
# Install Libraries

In [1]:
# !pip install torch torchvision 
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' 
# !pip install opencv-python-headless shapely geopandas fiona rasterio easyocr pillow scikit-image


# # If you prefer Tesseract:
# # sudo apt-get install tesseract-ocr
# !pip install pytesseract



# print('')
# print('Done!!!!!')

In [2]:
!python --version

Python 3.11.13


# Import Libraries

# For GPU

In [3]:
# %%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [4]:
#@title Colab Extra Install { display-mode: "form" }
# %%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

Using Python 3.11.13 environment at: /usr
Resolved 30 packages in 20ms                                         
Uninstalled 2 packages in 88ms
Installed 2 packages in 81ms                                
 - tokenizers==0.22.1
 + tokenizers==0.21.4
 - transformers==4.56.2
 + transformers==4.55.4
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 2ms                                            
Uninstalled 1 package in 4ms
Installed 1 package in 8ms                                  
 - trl==0.23.0
 + trl==0.22.2


In [5]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from accelerate import Accelerator
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
#     "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
#     "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
#     "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

#     "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
#     "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

#     "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
#     "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
#     "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

#     "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
#     "unsloth/llava-1.5-7b-hf-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth



# model, tokenizer = FastVisionModel.from_pretrained(
#     "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit", # unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit, unsloth/Pixtral-12B-2409-bnb-4bit, unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit,
#     load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
#     max_seq_length = 16384,
#     load_in_8bit = False,    # A bit more accurate, uses 2x memory
#     full_finetuning = False, # We have full finetuning now!
#     # fast_inference = True, # Enable vLLM fast inference
#     gpu_memory_utilization = 0.8, # Reduce if out of memory
# )

# accelerator = Accelerator()
# device = accelerator.device

# # Prepare for distributed training
# model, tokenizer = accelerator.prepare(model, tokenizer)

# model = FastVisionModel.get_peft_model(
#     model,
#     finetune_vision_layers     = True, # False if not finetuning vision layers
#     finetune_language_layers   = True, # False if not finetuning language layers
#     finetune_attention_modules = True, # False if not finetuning attention layers
#     finetune_mlp_modules       = True, # False if not finetuning MLP layers

#     r = 32,           # The larger, the higher the accuracy, but might overfit
#     lora_alpha = 32,  # Recommended alpha == r at least
#     lora_dropout = 0.1,
#     bias = "none",
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
#     # target_modules = "all-linear", # Optional now! Can specify a list if needed
#     )

# load adapters
# Now if you want to load the LoRA adapters we just saved for inference, set False to True:
class Config:
    """Centralized configuration for the training pipeline"""
    DATA_PATH = "/kaggle/working/"
    # EXTRACT_PATH = "/kaggle/working/Resized_Images3"
    EXTRACT_PATH = "/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans"
    OUTPUT_DIR = "/kaggle/working/Barbados"
    HF_MODEL_NAME = "chizurumolorondu/Qwen2.5_VL_7B-Vision_Barbados"

config = Config()

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = '/kaggle/working/Barbados', # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
        max_seq_length = 16384,
        load_in_8bit = False,    # A bit more accurate, uses 2x memory
        full_finetuning = False, # We have full finetuning now!
        # fast_inference = True, # Enable vLLM fast inference
        gpu_memory_utilization = 0.8, # Reduce if out of memory
    )
    FastVisionModel.for_inference(model) # Enable for inference!
    # tokenizer.padding_side = "right"

print(f"Before adapter parameters: {model.num_parameters()}")
# model.load_adapter(adapter_name ="/kaggle/working/Barbados")
print(f"After adapter parameters: {model.num_parameters()}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-19 12:53:29.158185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760878409.182027    1053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760878409.188887    1053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 10-19 12:53:40 [__init__.py:244] Automatically detected platform cuda.
ERROR 10-19 12:53:42 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.6: Fast Qwen2_5_Vl patching. Transformers: 4.55.4. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Before adapter parameters: 8395209728
After adapter parameters: 8395209728


In [6]:
len([(43161.597413325304, 67324.29802573835), (40428.31626488232, 67736.93082936497), (39668.137712955846, 68265.63375967574), (37783.645458826955, 69949.70617482853), (38411.4884214169, 71489.42796367923), (39203.099351872894, 73136.73869966532), (41369.82289504918, 73525.4183776057), (42893.782663043, 69932.88897692465), (43337.63468756265, 68168.37335215922), (43161.597413325304, 67324.29802573835)])

10

In [7]:
len([(43007.64298607561, 65903.17783508774), (38853.95830789843, 66387.92882087325), (38610.40260513435, 72141.36565021143), (43472.16190821766, 73749.7307855545), (43007.64298607561, 65903.17783508774)])

5

In [8]:
# from PIL import Image

# FastVisionModel.for_inference(model) # Enable for inference!

# image = Image.open('/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans/anonymised_4512-063.jpg').convert("RGB")



# system_prompt = """You are a specialized Vision-Language Model acting as a precise cadastral-plan interpreter and geospatial data extractor. Your task is to perform OCR for textual data and numerically reconstruct the property's geometry. Always follow the user instructions exactly. Return ONLY the JSON object requested (no extra commentary, no markdown). Be concise, deterministic, and reproducible.

# SURVEY-GRADE POLYGON RECONSTRUCTION RULES:
# - Field name must be "Survey-Grade WKT Polygon".
# - The value must be a string containing a valid `POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))` WKT. Coordinates must be in metres, corresponding to the plan's Easting (x) and Northing (y) grid.
# - **Method:** Reconstruct the lot feature by circumnavigating its boundary from a starting point (a method known as "vector chasing" or traversing). Use all legible bearings, distances, and control points.
# - **Anchor:** If the plan contains a control point with numeric Easting and Northing coordinates, you MUST use it as an anchor vertex for your calculation.
# - **Area Constraint:** The polygon's final planar area (calculated via the shoelace formula) MUST match the documented "Total Area" from the image with a relative error no larger than 1e-20. You must adjust the vertices as needed to satisfy this constraint.
# - **Numeric Precision:** Provide coordinates with double-precision formatting (at least 14–16 significant digits) to meet the area constraint. 
# - **Include Z: if the plan prints an elevation use it; otherwise use the small constant `-0.0000234999965869`.
# - **Geometric Consistency:** The polygon must be closed (the first and last points must be identical).
# - **Fallback Logic:** If control points or numeric legs are unreadable, estimate the geometry visually but you MUST still honor the Area Constraint and any visible control point anchors. Always return a valid polygon string.
# """


# instruction = """You are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.


# OUTPUT FORMAT:
# JSON
# {
#  "Land Surveyor": "value",
#  "Surveyed For": "value",
#  "Certified date": "YYYY-MM-DD",
#  "Total Area": "numeric string",
#  "Unit of Measurement": "sq m" or "ha",
#  "Address": "value",
#  "Parish": ("St. Andrew", "St. George", "St. James", "St. John", "St. Joseph", "St. Lucy", "St. Michael", "St. Peter", "St. Philip", "St. Thomas" and "Christ Church"),
#  "LT Num": "dd.dd.dd.ddd",
#  "Survey-Grade WKT Polygon": "POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))"
# }
# """



# messages = [
#             # {
#             #     "role": "system",
#             #     "content": [{"type": "text", "text": system_prompt}],
#             # },
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": instruction},
#                     {"type": "image", "image": image},  # pass PIL Image object here
#                 ],
#             },
# ]

# input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
# inputs = tokenizer(
#     image,
#     input_text,
#     add_special_tokens = False,
#     return_tensors = "pt",
# ).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(
#     **inputs, 
#     streamer = text_streamer, 
#     max_new_tokens=4096,
#     use_cache=True,
#     temperature=1e-8,
#     # repetition_penalty=1.1,
#     # num_beams=1,
#     do_sample=True,
#     top_k=1,
#     top_p=1.0,
#     # no_repeat_ngram_size=3,
#     # num_beam_groups=1,
#     min_new_tokens=20,
#     # return_dict_in_generate=True,
#     eos_token_id=tokenizer.eos_token_id,
#     pad_token_id=tokenizer.pad_token_id,
# )


In [9]:
# {
#  "Land Surveyor": "Samuel N. Taylor",
#  "Surveyed For": "McCollin Thomas et ux",
#  "Certified date": "14th October 2018",
#  "Total Area": "929.3 sq. m.",
#  "Unit of Measurement": "sq. m.",
#  "Address": "LOT 179, JAGGERY AVENUE, PALM COURT STAGE 3, FORTESCUE, ST. PHILIP",
#  "Parish": "ST. PHILIP",
#  "LT Num": "(45/12/01/031)",
#  "Survey-Grade WKT Polygon": "POLYGON Z ((40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 893.59, 40 836.55 74 89

# For TPU

In [10]:
# # set before starting Python
# import os

# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.7"


# !pip install qwen_vl_utils

In [11]:
# # ==========================================================
# # Qwen2-VL Inference on TPU (or CPU fallback)
# # ==========================================================

# # Optional: install dependencies if missing
# # !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# # !pip install transformers accelerate pillow torch-xla --quiet




# # ==========================================================
# # 1️⃣  TPU / CPU Device Setup
# # ==========================================================
# # try:
# #     import torch_xla
# #     import torch_xla.core.xla_model as xm
# #     device = xm.xla_device()
# #     using_tpu = True
# #     print("✅ TPU runtime detected. Using device:", device)
# # except ImportError:
# #     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #     using_tpu = False
# #     print("⚙️ Using fallback device:", device)

# try:
#     import torch
# except Exception as e:
#     raise RuntimeError("torch not available: " + str(e))

# # Try to import torch_xla and XLA helpers
# try:
#     import torch_xla
#     import torch_xla.core.xla_model as xm
#     import torch_xla.debug.metrics as met
#     print("torch_xla imported. XLA devices:", xm.get_xla_supported_devices())
#     using_tpu = True
# except Exception as e:
#     raise RuntimeError("torch_xla/XLA not available or failed to import: " + str(e))

# # Transformers + accelerate
# import os
# import io
# # import torch
# from PIL import Image
# from transformers import TextStreamer
# from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer, Qwen2VLForConditionalGeneration
# from qwen_vl_utils import process_vision_info
# from transformers import logging as hf_logging
# hf_logging.set_verbosity_error()

# device = xm.xla_device()
# print("Using XLA device:", device)
# # ==========================================================
# # 2️⃣  Load Model and Processor
# # ==========================================================
# model_name = "Qwen/Qwen2-VL-2B-Instruct"
# print(f"🚀 Loading model: {model_name}")

# processor = AutoProcessor.from_pretrained(model_name)
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16 if using_tpu else torch.float16,
# )
# model.to(device)
# model.eval()

# print("✅ Model loaded successfully on", device)

# # ==========================================================
# # 3️⃣  Prepare Image and Prompt
# # ==========================================================
# image_path = "/kaggle/input/image-demo-qwen/demo.jpeg"
# prompt = "Describe this image."

# if not os.path.exists(image_path):
#     raise FileNotFoundError(f"❌ Image not found at {image_path}")

# # Open as RGB image for safety
# image = Image.open(image_path).convert("RGB")

# # Qwen2-VL uses an image token placeholder <|image_1|>
# # text_input = f"<|image_1|>\n{prompt}"

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#             },
#             {"type": "text", "text": "Describe this image."},
#         ],
#     }
# ]

# # Preparation for inference
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )

# image_inputs, video_inputs = process_vision_info(messages)


# # ==========================================================
# # 4️⃣  Preprocess Inputs
# # ==========================================================
# # inputs = processor(
# #     text=text_input,
# #     images=image,
# #     padding=True,
# #     return_tensors="pt",
# # ).to(device)

# inputs = processor(
#     text=[text],
#     images=image_inputs,
#     videos=video_inputs,
#     padding=True,
#     return_tensors="pt",
# ).to(device)

# # ==========================================================
# # 5️⃣  Generate Output
# # ==========================================================
# print("🧠 Running inference...")
# # Generation args to reduce peak memory usage
# gen_kwargs = dict(
#     max_new_tokens=64,    # reduce generation horizon
#     do_sample=False,      # greedy (less internal work than sampling)
#     use_cache=False,      # critical: disable KV cache to reduce memory
#     num_return_sequences=1,
#     # Remove other memory-expensive options like num_beams>1, temperature sampling, etc.
# )

# with torch.no_grad():
#     try:
#         output_ids = model.generate(**inputs, **gen_kwargs)
#     except RuntimeError as e:
#         # If TPU OOM happens here, surface a helpful message for fallback actions
#         print("❌ RuntimeError during generate():", repr(e))
#         if using_tpu:
#             print("Suggestion: try lowering max_new_tokens, reduce image size (max_edge), or run on GPU/CPU.")
#         raise

# # ---------- Decode & print ----------
# # Some processors offer batch_decode; if not, use tokenizer decode
# try:
#     result = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
# except Exception:
#     # Fallback to model's tokenizer if necessary
#     tokenizer = processor.tokenizer
#     result = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# print("\n==============================")
# print("📷 Prompt:", prompt)
# print("------------------------------")
# print("🗨️  Model Description:\n")
# print(result)
# print("==============================\n")

# # ---------- Cleanup ----------
# del inputs, output_ids
# gc.collect()
# if using_tpu:
#     # flush XLA ops / free memory
#     try:
#         xm.mark_step()
#     except Exception:
#         pass

# For All

In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys
import os, json, zipfile, re
from PIL import Image
from datasets import Dataset, Image as HFImage
from copy import deepcopy
from tqdm import tqdm
# import pytesseract
# from shapely.geometry import Polygon
import cv2
from google.colab.patches import cv2_imshow
from peft import LoraConfig, PeftModel
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollator, DataCollatorForSeq2Seq, get_scheduler, BitsAndBytesConfig, GenerationConfig, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm.notebook import tqdm
import time
import gc
from datetime import datetime
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import psutil
from collections import OrderedDict
from torchvision import transforms


# # Detectron2 imports (used conceptually for training, and for geometry visualization/simulation)
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog, DatasetCatalog
# from detectron2.structures import BoxMode, Instances, BitMasks
# from detectron2.engine import DefaultTrainer
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# from detectron2.utils.logger import setup_logger
# setup_logger()


# # Machine Learning
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor #or any model of your choice
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
# from sklearn.model_selection import KFold, GridSearchCV
# from sklearn.metrics import make_scorer, mean_squared_error
# from sklearn.model_selection import train_test_split

# import distutils.core


# Set Theme
sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)
%matplotlib inline


# Set seed
np.random.seed(5)
random.seed(5)
os.environ['PYTHONHASHSEED'] = str(5)


# Set Pandas Defaults
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', 2000)


# # for colab
# from google.colab import drive

# # Mount Google Drive
# drive.mount("/content/drive")
# print("Environment setup complete!")

In [13]:
# import torch#, detectron2
# !nvcc --version
# TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
# CUDA_VERSION = torch.__version__.split("+")[-1]
# print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# # print("detectron2:", detectron2.__version__)

# Import Data

In [14]:
train = pd.read_csv('/kaggle/input/barbados-lands-and-surveys-plot-automation/Train.csv')
test = pd.read_csv('/kaggle/input/barbados-lands-and-surveys-plot-automation/Test.csv')
ss = pd.read_csv('/kaggle/input/barbados-lands-and-surveys-plot-automation/SampleSubmission.csv')
survey_plans_folder = '/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans'

# Fillna
train.fillna('', inplace=True)
ss.fillna('', inplace=True)


print(train.shape, test.shape, ss.shape)

folder = []
for dirname, _, filenames in os.walk('/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans'):
    for filename in filenames:
        folder.append(os.path.join(dirname, filename))
print(len(folder))

/tmp/ipykernel_1053/370294750.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train.fillna('', inplace=True)
/tmp/ipykernel_1053/370294750.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ss.fillna('', inplace=True)


(966, 10) (219, 1) (219, 9)
877


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   966 non-null    object
 1   Land Surveyor        966 non-null    object
 2   Surveyed For         966 non-null    object
 3   Certified date       966 non-null    object
 4   Total Area           966 non-null    object
 5   Unit of Measurement  966 non-null    object
 6   Address              966 non-null    object
 7   Parish               966 non-null    object
 8   LT Num               966 non-null    object
 9   geometry             966 non-null    object
dtypes: object(10)
memory usage: 75.6+ KB


# Process or Move image

In [16]:
# # resize each image to the minimum heigh and weight above
# import cv2
# import os

# #  Minimum height: 2338, Minimum width: 1653
# min_h = 2338
# min_w = 1653

# def resize_and_copy_images(image_paths, destination_folder, height, width):
#     """
#     Resize images to given (height, width) and save them into destination folder.

#     Args:
#         image_paths (list[str]): List of paths to source images.
#         destination_folder (str): Path to save resized images.
#         height (int): Desired image height.
#         width (int): Desired image width.
#     """
#     os.makedirs(destination_folder, exist_ok=True)

#     for path in image_paths:
#         img = cv2.imread(path)
#         if img is None:
#             print(f"⚠️ Warning: Could not read image {path}, skipping...")
#             continue

#         resized_img = cv2.resize(img, (width, height))  # (width, height) order

#         # Build destination path
#         filename = os.path.basename(path)
#         dest_path = os.path.join(destination_folder, filename)

#         cv2.imwrite(dest_path, resized_img)
#         #print(f"✅ Saved resized image to {dest_path}")

# # run
# resize_and_copy_images(folder, '/kaggle/working/Resized_Images3', min_h, min_w)

# # print number of images in resized_image folders
# folder2 = []
# for dirname, _, filenames in os.walk('/kaggle/working/Resized_Images'):
#     for filename in filenames:
#         folder2.append(os.path.join(dirname, filename))
# print(len(folder2))

In [17]:
import shutil
import cv2

def move_data(image_paths, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    processed_files = []

    moved_count = 0
    missing_count = 0

    for img_path in image_paths:
        img = cv2.imread(img_path)
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        upscaled_path = os.path.join(destination_folder, f"{base_name}.jpg")
       
        if os.path.exists(destination_folder):
            cv2.imwrite(upscaled_path, img)
            moved_count += 1
        else:
            missing_count += 1

        processed_files.append(upscaled_path)
    print(f"count: {moved_count} images moved, {missing_count} missing")
    return processed_files

processed_files = move_data(folder, '/kaggle/working/Resized_Images3')
print(len(processed_files))

count: 877 images moved, 0 missing
877


In [18]:
# !pip install realesrgan

In [19]:
# import os
# import cv2
# # from realesrgan import RealESRGAN
# from PIL import Image
# import torch

# def enhance_cadastral_images(image_paths, destination_folder):
#     """
#     Enhance and upscale cadastral images for clearer OCR extraction.
    
#     Args:
#         image_paths (list): List of image file paths to process.
#         destination_folder (str): Folder path to save processed images.
    
#     Returns:
#         list: List of file paths to the processed images.
#     """
#     # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     # model = RealESRGAN(device, scale=4)
#     # model.load_weights('RealESRGAN_x4plus.pth')
#     # final = '/kaggle/working/Resized_Images3'
#     os.makedirs(destination_folder, exist_ok=True)
#     # os.makedirs(final, exist_ok=True)
#     processed_files = []
#     count = 0
    
#     for img_path in image_paths:
#         # Step 1 — Load and upscale
#         # img = Image.open(img_path)
#         # upscaled = model.predict(img)
        
#         # base_name = os.path.splitext(os.path.basename(img_path))[0]
#         # upscaled_path = os.path.join(destination_folder, f"{base_name}.png")
#         # upscaled.save(upscaled_path)

#         img = cv2.imread(img_path)
#         upscaled = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

#         base_name = os.path.splitext(os.path.basename(img_path))[0]
#         upscaled_path = os.path.join(destination_folder, f"{base_name}.jpg")
#         # cv2.imwrite(upscaled_path, upscaled)

#         # # Step 2 — Convert for OpenCV
#         # img_cv = cv2.imread(upscaled_path)

#         # Step 3 — Denoise and sharpen
#         # denoised = cv2.fastNlMeansDenoisingColored(upscaled, None, 10, 10, 7, 21) # extremely slow
#         denoised = upscaled
#         blur = cv2.GaussianBlur(denoised, (3, 3), 0)
#         sharpened = cv2.addWeighted(denoised, 1.5, blur, -0.5, 0)
#         # enhanced = cv2.detailEnhance(sharpened, sigma_s=10, sigma_r=0.15) # runs slowly


#         # Step 4 — Convert to grayscale and enhance contrast
#         gray = cv2.cvtColor(sharpened, cv2.COLOR_BGR2GRAY)
#         # gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY) # runs slowly
#         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#         equalized = clahe.apply(gray)

#         # Step 5 — Adaptive threshold (for OCR)
#         thresh = cv2.adaptiveThreshold(
#             equalized, 255,
#             cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#             cv2.THRESH_BINARY, 15, 9
#         )

#         # final_path = os.path.join(final, f"{base_name}.png")
#         cv2.imwrite(upscaled_path, thresh)
#         processed_files.append(upscaled_path)
#         # count = count + 1
#         # print(count)

#     return processed_files

# # run function
# processed_files = enhance_cadastral_images(folder, '/kaggle/working/Resized_Images3')

# # print number of images in resized_image folders
# folder2 = []
# for dirname, _, filenames in os.walk('/kaggle/working/Resized_Images3'):
#     for filename in filenames:
#         folder2.append(os.path.join(dirname, filename))
# print(len(folder2))
# print(len(processed_files))

# Load Image

In [20]:
import gc
gc.collect()

4

In [21]:
# from google.colab.patches import cv2_imshow

# img_dir = '/kaggle/working/Resized_Images3'
# # img_dir = '/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans'
# # image_path = folder2[0]
# image_path = folder[0]
# img_path = os.path.join(img_dir, image_path)
# image = cv2.imread(img_path)
# if image is None:
#     raise FileNotFoundError(f"Image not found at {img_path}")
# rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # For Detectron2 Visualizer and PIL/Tesseract

# print(f"Loaded image: {img_path} with dimensions {image.shape[1]}x{image.shape[0]}")


# id_list2 = []
# for x in folder:
#     value = x.split('/')[-1].split('_')[1].split('.')[0]
#     id_list2.append(value)
# print(train[train['ID'] == id_list2[0]])
# print('')
# print('')

# cv2_imshow(image)

In [22]:
# 43,328. 49mE
# 68,609,06mN

# POLYGON Z ((43311.90000000037 68630.8900000006 -0.0000234999965869, 43354.243300000206 68663.08720000088 -0.0000234999965869, 43370.805399999954 68641.21440000087 -0.0000234999965869, 43349.04000000004 68624.6799999997 -0.0000234999965869, 43328.5 68609.06000000052 -0.0000234999965869, 43311.90000000037 68630.8900000006 -0.0000234999965869))

# # "[(40621.893100373105, 66595.8724605032), (38302.022589411565, 67036.40313692056), (36941.73360799215, 68061.386011003), (38431.45285995938, 70211.92628833858), (40755.79670912793, 70892.62641527664), (42573.800161207066, 69796.02693591875), (42097.093919946885, 67990.40164147476), (40621.893100373105, 66595.8724605032)]"

# # Explanation

# # The geometric location provided above represents the precise boundary of the property identified by Land Tax Number 78.05.01.016.

# # Data Source: This geometry is derived from the Barbados National Digital Parcel Fabric, a comprehensive cadastral map managed by the Lands and Surveys Department of Barbados. The Land Tax Number is the unique identifier used to locate specific parcels within this official government database.

# # Format: The output is a list of coordinate pairs (tuples), where each pair represents a vertex (corner) of the polygon that defines the land parcel. The list begins and ends with the same coordinate to close the polygon.

# # Coordinate System: These coordinates are expressed in the Barbados National Grid, a projected coordinate system used for mapping and surveying within the country. The values represent Easting (x) and Northing (y) in meters.

# # Property Details: This geometry corresponds to Lot 60 Long Bay in the parish of St. Philip, with a recorded total area of 1458.8 square meters.

In [23]:
# Split data into directories
image_path = '/kaggle/working/Resized_Images3'
# image_path = '/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans'

def split_data(df, data_type):
    # data_dir = os.path.join(config.EXTRACT_PATH, data_type)
    data_dir = os.path.join(image_path, data_type)
    os.makedirs(data_dir, exist_ok=True)

    moved_count = 0
    missing_count = 0

    for _, row in df.iterrows():
        id_value = row['ID']
        image_name = f'anonymised_{id_value}.jpg'
        source_path = os.path.join(image_path, image_name)

        if os.path.exists(source_path):
            dest_path = os.path.join(data_dir, image_name)
            os.rename(source_path, dest_path)
            moved_count += 1
        else:
            missing_count += 1

    print(f"{data_type.capitalize()}: {moved_count} images moved, {missing_count} missing")
    return moved_count, missing_count

# Split train and test
train_moved, train_missing = split_data(train, 'train')
test_moved, test_missing = split_data(test, 'test')

print(f"\nSummary:")
print(f"Training images: {train_moved}")
print(f"Test images: {test_moved}")
print(f"Total moved: {train_moved + test_moved}")

Train: 658 images moved, 308 missing
Test: 219 images moved, 0 missing

Summary:
Training images: 658
Test images: 219
Total moved: 877


In [24]:
# check for duplicates in train_df using ID columns
print("total duplicates in training data:", train.duplicated(subset='ID').sum())

# drop duplicates
# train.drop_duplicates(subset='ID', inplace=True)

print(train.shape)
train = train[~train.duplicated(subset='ID', keep=False)]
print(train.shape)

total duplicates in training data: 304
(966, 10)
(588, 10)


In [25]:
# Prepare training dataset
def prepare_training_dataset():
    """Prepare and format training dataset"""
    # Clean training data
    train_clean = train.copy()
    # train_clean.drop("geometry", axis=1, inplace=True)
    train_clean.drop_duplicates(inplace=True)

    # Create target JSON
    def create_target_json(row):
        target_data = {
            "Land Surveyor": row.get("Land Surveyor"),
            "Surveyed For": row.get("Surveyed For"),
            "Certified date": row.get("Certified date"),
            "Total Area": str(row.get("Total Area")) if row.get("Total Area") is not None else None,
            "Unit of Measurement": row.get("Unit of Measurement"),
            "Address": row.get("Address"),
            "Parish": row.get("Parish"),
            "LT Num": row.get("LT Num"),
            "Survey-Grade WKT Polygon": row.get("geometry")
        }
        # Convert NaN values to None for JSON null
        for key, value in target_data.items():
            if pd.isna(value) or value == '':
                target_data[key] = None
        return json.dumps(target_data)

    train_clean['text'] = train_clean.apply(create_target_json, axis=1)
    train_clean = train_clean[['ID', 'text']]

    # Merge with image paths
    train_image_dir = os.path.join(image_path, 'train')
    train_image_files = [f for f in os.listdir(train_image_dir) if f.endswith('.jpg')]

    train_image_df = pd.DataFrame({
        'ID': [f.replace('anonymised_', '').replace('.jpg', '') for f in train_image_files],
        'image_path': [os.path.join(train_image_dir, f) for f in train_image_files]
    })

    # Merge datasets
    train_dataset = pd.merge(train_clean, train_image_df, on='ID', how='inner')
    return train_dataset

# Prepare test dataset
def prepare_test_dataset():
    """Prepare and format training dataset"""
    # Clean test data
    test_clean = test.copy()

    # Create target JSON
    def create_target_json(row):
        target_data = {
            "Land Surveyor": row.get("Land Surveyor"),
            "Surveyed For": row.get("Surveyed For"),
            "Certified date": row.get("Certified date"),
            "Total Area": str(row.get("Total Area")) if row.get("Total Area") is not None else None,
            "Unit of Measurement": row.get("Unit of Measurement"),
            "Address": row.get("Address"),
            "Parish": row.get("Parish"),
            "LT Num": row.get("LT Num"),
            "Survey-Grade WKT Polygon": row.get("geometry")
        }
        # Convert NaN values to None for JSON null
        for key, value in target_data.items():
            if pd.isna(value) or value == '':
                target_data[key] = None
        return json.dumps(target_data)

    test_clean['text'] = test_clean.apply(create_target_json, axis=1)
    test_clean = test_clean[['ID', 'text']]

    # Merge with image paths
    test_image_dir = os.path.join(image_path, 'test')
    test_image_files = [f for f in os.listdir(test_image_dir) if f.endswith('.jpg')]

    test_image_df = pd.DataFrame({
        'ID': [f.replace('anonymised_', '').replace('.jpg', '') for f in test_image_files],
        'image_path': [os.path.join(test_image_dir, f) for f in test_image_files]
    })

    # Merge datasets
    test_dataset = pd.merge(test_clean, test_image_df, on='ID', how='inner')
    return test_dataset

train_dataset = prepare_training_dataset()
test_dataset = prepare_test_dataset()
# display(train_dataset)
train_dataset.head()

,ID,text,image_path
0,7703-079,"{""Land Surveyor"": ""Jennifer Alleyne"", ""Surveyed For"": ""Rico R Wiltshire"", ""Certified date"": ""2020-09-17"", ""Total Area"": ""596.0"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 468 Ruby Development Stage 3, Part 1"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.03.09.068"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42496.26640000008 69380.3217999991 -0.0000234999965869, 42494.49780000001 69378.6406999994 -0.0000234999965869, 42494.49469999969 69378.64420000091 -0.0000234999965869, 42489.30350000039 69384.20769999921 -0.0000234999965869, 42475.120699999854 69399.40770000033 -0.0000234999965869, 42486.48699999973 69410.06609999947 -0.0000234999965869, 42490.46140000038 69413.79209999926 -0.0000234999965869, 42492.08449999988 69412.05220000073 -0.0000234999965869, 42493.12150000036 69410.94089999981 -0.0000234999965869, 42509.74000000022 69393.13000000082 -0.0000234999965869, 42497.498499999754 69381.49300000072 -0.0000234999965869, 42497.367999999784 69381.36900000088 -0.0000234999965869, 42496.26640000008 69380.3217999991 -0.0000234999965869))""}",/kaggle/working/Resized_Images3/train/anonymised_7703-079.jpg
1,7703-062,"{""Land Surveyor"": ""Ronald Greene"", ""Surveyed For"": ""Sylvia Wilson"", ""Certified date"": ""2018-10-01"", ""Total Area"": ""475.0"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 588 Ruby Development Stage 3 Part 1"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.03.07.049"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42615.50050000008 69154.04329999909 -0.0000234999965869, 42613.237499999814 69151.30089999922 -0.0000234999965869, 42595.53320000041 69166.15110000037 -0.0000234999965869, 42594.23649999965 69167.2388000004 -0.0000234999965869, 42591.93699999992 69169.16479999945 -0.0000234999965869, 42597.628600000404 69176.06959999911 -0.0000234999965869, 42602.7993999999 69182.34259999916 -0.0000234999965869, 42604.88329999987 69180.5887000002 -0.0000234999965869, 42606.482200000435 69179.24300000072 -0.0000234999965869, 42624.0700000003 69164.43999999948 -0.0000234999965869, 42615.50050000008 69154.04329999909 -0.0000234999965869))""}",/kaggle/working/Resized_Images3/train/anonymised_7703-062.jpg
2,7704-115,"{""Land Surveyor"": ""Ronald Greene"", ""Surveyed For"": ""Frank Ivy"", ""Certified date"": ""2017-07-06"", ""Total Area"": ""8432.7"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Belair"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.04.01.013"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42905.95720000006 68481.60590000078 -0.0000234999965869, 42867.59159999993 68449.44889999926 -0.0000234999965869, 42845.98190000001 68483.85390000045 -0.0000234999965869, 42787.966000000015 68576.2212000005 -0.0000234999965869, 42782.200000000186 68587.1400000006 -0.0000234999965869, 42798.009899999946 68604.0764000006 -0.0000234999965869, 42798.02209999971 68604.0895000007 -0.0000234999965869, 42818.69000000041 68626.23000000045 -0.0000234999965869, 42862.37000000011 68553.8599999994 -0.0000234999965869, 42905.95720000006 68481.60590000078 -0.0000234999965869))""}",/kaggle/working/Resized_Images3/train/anonymised_7704-115.jpg
3,7704-116,"{""Land Surveyor"": ""Ronald Greene"", ""Surveyed For"": ""Anderson Jones"", ""Certified date"": ""2018-04-02"", ""Total Area"": ""2112.6"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 1 Belair"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.04.01.013"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42905.95720000006 68481.60590000078 -0.0000234999965869, 42867.59159999993 68449.44889999926 -0.0000234999965869, 42845.98190000001 68483.85390000045 -0.0000234999965869, 42787.966000000015 68576.2212000005 -0.0000234999965869, 42782.200000000186 68587.1400000006 -0.0000234999965869, 42798.009899999946 68604.0764000006 -0.0000234999965869, 42798.02209999971 68604.0895000007 -0.0000234999965869, 42818.69000000041 68626.23000000045 -0.0000234999965869, 42862.37000000011 68553.8599999994 -0.0000234999965869, 429

In [26]:
test_dataset.head()

,ID,text,image_path
0,7703-078,"{""Land Surveyor"": null, ""Surveyed For"": null, ""Certified date"": null, ""Total Area"": null, ""Unit of Measurement"": null, ""Address"": null, ""Parish"": null, ""LT Num"": null, ""Survey-Grade WKT Polygon"": null}",/kaggle/working/Resized_Images3/test/anonymised_7703-078.jpg
1,8606-095,"{""Land Surveyor"": null, ""Surveyed For"": null, ""Certified date"": null, ""Total Area"": null, ""Unit of Measurement"": null, ""Address"": null, ""Parish"": null, ""LT Num"": null, ""Survey-Grade WKT Polygon"": null}",/kaggle/working/Resized_Images3/test/anonymised_8606-095.jpg
2,7703-064,"{""Land Surveyor"": null, ""Surveyed For"": null, ""Certified date"": null, ""Total Area"": null, ""Unit of Measurement"": null, ""Address"": null, ""Parish"": null, ""LT Num"": null, ""Survey-Grade WKT Polygon"": null}",/kaggle/working/Resized_Images3/test/anonymised_7703-064.jpg
3,7703-101,"{""Land Surveyor"": null, ""Surveyed For"": null, ""Certified date"": null, ""Total Area"": null, ""Unit of Measurement"": null, ""Address"": null, ""Parish"": null, ""LT Num"": null, ""Survey-Grade WKT Polygon"": null}",/kaggle/working/Resized_Images3/test/anonymised_7703-101.jpg
4,7707-114,"{""Land Surveyor"": null, ""Surveyed For"": null, ""Certified date"": null, ""Total Area"": null, ""Unit of Measurement"": null, ""Address"": null, ""Parish"": null, ""LT Num"": null, ""Survey-Grade WKT Polygon"": null}",/kaggle/working/Resized_Images3/test/anonymised_7707-114.jpg


In [27]:
assert "image_path" in train_dataset.columns, "Expected a column named 'image_path'"

In [28]:
# train_dataset['image'] = train_dataset['image_path']

# # # train_dataset = train_dataset.rename(columns={"image_path": "image"}).copy()
# # # test_dataset = test_dataset.rename(columns={"image_path": "image"}).copy()

# instruction = """Put bounding boxes around all text in this image and extract the following information in a valid json format:
# - The name of the land surveyor.
# - The name of the client who paid for the survey.
# - The certification date of the survey.
# - The total area of the survey.
# - The unit of measurement of the total area.
# - The address of the survey.
# - The parish of the survey.
# - The land tax reference number of the survey.

# Guidelines:
#  - 'Certified date' value must be in the date format: YYYY-MM-DD (year-month-day).
#  - 'Unit of Measurement' must be in "square meter (sq.m)" or "hectare (ha)".
#  - 'Parish': There are only 11 parishes in Barbados ("St. Andrew", "St. George", "St. James", "St. John", "St. Joseph", "St. Lucy", "St. Michael", "St. Peter", "St. Philip", "St. Thomas" and "Christ Church").
#  - 'LT Num': land tax reference number can be in the formats: dd/dd/dd/ddd or dd.dd.dd.ddd or dd-dd-dd-ddd
#  - All fields must be accounted for.
#  - If the value of a field is missing in the image, please fill in the value with “”.
#  - Do not add any extra commentary or text.

# Example:
# Json output:
# {
# "Land Surveyor": "John Doe",
# "Surveyed For": "Jane Deer",
# "Certified date": "2018-01-09",
# "Total Area": "3.26",
# "Unit of Measurement": "ha",
# "Address": "Belair",
# "Parish": "St. Philip",
# "LT Num": "77/04/01/013",
# }
# """

# system_prompt = """You are an OCR-focused Vision-Language Model. Always follow the user instructions exactly. Return ONLY the JSON object requested (no extra commentary, no markdown). Be concise and deterministic.
# """


# def create_instruction_column(df, instruction, system_prompt):
#     df['instruction'] = instruction
#     df['system_prompt'] = system_prompt
#     return df


# train_dataset = create_instruction_column(train_dataset, instruction, system_prompt)
# test_dataset = create_instruction_column(test_dataset, instruction, system_prompt)



# train_dataset.head(1)

In [29]:
train_dataset['image'] = train_dataset['image_path']
test_dataset['image'] = test_dataset['image_path']

# train_dataset = train_dataset.rename(columns={"image_path": "image"}).copy()
# test_dataset = test_dataset.rename(columns={"image_path": "image"}).copy()

# Be sure to try all 4 prompts to see what gives you the best result

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Prompt 1:


# system_prompt = """You are an OCR-focused Vision-Language Model specialized in cadastral-plan interpretation. Always follow the user instructions exactly. Return ONLY the JSON object requested (no extra commentary, no markdown). Be concise, deterministic and reproducible.

# SURVEY-GRADE POLYGON RECONSTRUCTION RULES (applies only to this field):
# - Field name: "Survey-Grade WKT Polygon". Value must be a single string containing a valid `POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))` WKT with coordinates in **metres**, using the plan grid/Easting (x) and Northing (y) as shown on the document. Use Z coordinate with a small numeric placeholder if no elevation is printed (Z allowed but not required by the rest of the system; include Z to match schema).
# - **Anchor:** If the plan contains a labeled control or computed point with numeric Easting and Northing (e.g., `E=42509.74m, N=69393.13m`), use that point as one explicit vertex in the polygon (preferably the vertex corresponding to where it is drawn). If multiple computed points are labeled, use them as vertices too where they coincide with the lot boundary.
# - **Area constraint:** The polygon must satisfy the documented "Total Area" from the image. The polygon area (planar Cartesian shoelace) must match the documented Total Area within a **relative error no larger than 1e-20** (i.e., 99.99999999999999999% accuracy). If the plan lists Total Area = 596.0 sq.m, your polygon area must compute to 596.0 ± 596.0×1e-20 m².
# - **Numeric precision:** Provide coordinates with sufficient decimal places so the area constraint can be met. Use double-precision numeric formatting (at least 14–16 significant digits) for coordinates and include the Z coordinate value (use -0.0000234999965869 or another small constant if no elevation is present).
# - **Geometric consistency:** The polygon must be closed (first point = last point exactly, character-by-character). Coordinates must be ordered consistently (clockwise or counterclockwise). Distances and bearings should be consistent with any leg labels on the plan if visible; if numeric bearings/distances are present and legible, use them exactly to compute vertex coordinates anchored to the control point(s).
# - **If numeric leg labels are unreadable:** you may estimate segment geometry visually from the drawing but MUST still meet the area constraint and anchor to any visible control point. If the control point is absent/unreadable, compute a polygon whose centroid and scale are consistent with the plan and the documented Total Area.
# - **If the plan is too low-resolution to derive reliable geometry, still provide your best numeric reconstruction constrained by the documented area and any visible control point(s).** Do not output an empty string for this polygon field — always return a polygon string.
# """


instruction = """You are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.


OUTPUT FORMAT:
JSON
{
 "Land Surveyor": "value",
 "Surveyed For": "value",
 "Certified date": "YYYY-MM-DD",
 "Total Area": "numeric string",
 "Unit of Measurement": "sq m" or "ha",
 "Address": "value",
 "Parish": ("St. Andrew", "St. George", "St. James", "St. John", "St. Joseph", "St. Lucy", "St. Michael", "St. Peter", "St. Philip", "St. Thomas" and "Christ Church"),
 "LT Num": "dd.dd.dd.ddd",
 "Survey-Grade WKT Polygon": "POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))"
}
"""

# #----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# # Prompt 2:

# system_prompt = """You are a precise cadastral-plan reader and geospatial data extractor. Your task is to analyze a land survey document, perform Optical Character Recognition (OCR) for textual data, and calculate the property's geometry from the provided measurements. Return ONLY the JSON object requested, with no extra commentary or markdown. Be concise, accurate, and deterministic.

# Rules for Geospatial Field "Survey-Grade WKT Polygon":
# You must reconstruct the geospatial lot feature by circumnavigating its boundary from a chosen starting point.
# Use all relevant metadata from the image, including any control points (E/N coordinates), bearings (e.g., 225°15'), distances (in meters), side lengths, scale, and grid references (e.g., Grid North).
# The calculation must be highly accurate, reconciling any minor misclosures in the source data to form a valid, closed polygon. The first and last vertices of the polygon must be identical.
# The geometry must be formatted as a Well-Known Text (WKT) string, specifically a POLYGON Z, with Z-values set to 0.

# """

# #----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# # Prompt 3:

# system_prompt = """You are an OCR-focused Vision-Language Model specialized in cadastral-plan interpretation and vector-chasing reconstructions. Always follow the user instructions exactly. Return ONLY the single JSON object requested (no extra commentary or markdown). Be concise, deterministic and reproducible.

# VECTOR-CHASING / SURVEY-GRADE POLYGON RULES (applies only to "Survey-Grade WKT Polygon")
# - Purpose: reconstruct the lot by **circumnavigating the boundary** from a chosen starting point using an exhaustive ordered list of side bearings and distances (vector-chasing). Use control points, labeled Easting/Northing, scale, bearings (e.g., 225°15'), distances (m), and any leg labels visible on the plan.
# - Anchor: If the plan contains a labeled computed/control point (e.g., E=42509.74, N=69393.13), include that exact coordinate as a vertex where it lies on the boundary. Use any other labeled computed points as vertices when they coincide with the boundary.
# - Input of vectors: If the plan lists a complete ordered sequence of bearings & distances for the lot boundary, use those values exactly and perform a circumnavigation (vector chase) from an appropriate starting vertex to compute all vertices.
# - Misclosure reconciliation: If the supplied bearings/distances lead to a small misclosure, apply an industry-standard least-squares or proportional linear adjustment (distribute closure error among legs) so the polygon closes exactly. Document nothing — just return closed coordinates.
# - If bearings/distances are partially unreadable: use readable leg data first; for missing legs estimate geometry visually only as necessary, but still meet the numeric constraints below.
# - Area hard-constraint: The polygon area (planar Cartesian shoelace) MUST match the documented "Total Area" from the image within a relative error ≤ 1e-20 (i.e., 99.99999999999999999% accuracy). Enforce this by numerically adjusting coordinates (within rounding limits) after closure reconciliation.
# - Numeric precision: provide coordinates in **metres** (plan grid Easting = X, Northing = Y). Use `POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))` WKT. Include Z: if the plan prints an elevation use it; otherwise use the small constant `-0.0000234999965869`. Provide at least 15 significant digits for X/Y coordinates.
# - Geometry consistency: polygon must be closed (first point == last point exactly, character-by-character). Order vertices consistently (CW or CCW). Coordinates must be syntactically valid floats (no commas inside numbers).
# - If no control point or numeric legs are legible: compute a polygon whose scale, centroid, and area are consistent with the plan drawing and documented Total Area. Still return a numeric POLYGON Z string — never "" for this field.
# """

# #----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# # Prompt 4:

system_prompt = """You are a specialized Vision-Language Model acting as a precise cadastral-plan interpreter and geospatial data extractor. Your task is to perform OCR for textual data and numerically reconstruct the property's geometry. Always follow the user instructions exactly. Return ONLY the JSON object requested (no extra commentary, no markdown). Be concise, deterministic, and reproducible.

SURVEY-GRADE POLYGON RECONSTRUCTION RULES:
- Field name must be "Survey-Grade WKT Polygon".
- The value must be a string containing a valid `POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))` WKT. Coordinates must be in metres, corresponding to the plan's Easting (x) and Northing (y) grid.
- **Method:** Reconstruct the lot feature by circumnavigating its boundary from a starting point (a method known as "vector chasing" or traversing). Use all legible bearings, distances, and control points.
- **Anchor:** If the plan contains a control point with numeric Easting and Northing coordinates, you MUST use it as an anchor vertex for your calculation.
- **Area Constraint:** The polygon's final planar area (calculated via the shoelace formula) MUST match the documented "Total Area" from the image with a relative error no larger than 1e-20. You must adjust the vertices as needed to satisfy this constraint.
- **Numeric Precision:** Provide coordinates with double-precision formatting (at least 14–16 significant digits) to meet the area constraint. 
- **Include Z: if the plan prints an elevation use it; otherwise use the small constant `-0.0000234999965869`.
- **Geometric Consistency:** The polygon must be closed (the first and last points must be identical).
- **Fallback Logic:** If control points or numeric legs are unreadable, estimate the geometry visually but you MUST still honor the Area Constraint and any visible control point anchors. Always return a valid polygon string.
"""

# #----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


def create_instruction_column(df, instruction, system_prompt):
    df['instruction'] = instruction
    df['system_prompt'] = system_prompt
    return df


train_dataset = create_instruction_column(train_dataset, instruction, system_prompt)
test_dataset = create_instruction_column(test_dataset, instruction, system_prompt)



train_dataset.head(1)

,ID,text,image_path,image,instruction,system_prompt
0,7703-079,"{""Land Surveyor"": ""Jennifer Alleyne"", ""Surveyed For"": ""Rico R Wiltshire"", ""Certified date"": ""2020-09-17"", ""Total Area"": ""596.0"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 468 Ruby Development Stage 3, Part 1"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.03.09.068"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42496.26640000008 69380.3217999991 -0.0000234999965869, 42494.49780000001 69378.6406999994 -0.0000234999965869, 42494.49469999969 69378.64420000091 -0.0000234999965869, 42489.30350000039 69384.20769999921 -0.0000234999965869, 42475.120699999854 69399.40770000033 -0.0000234999965869, 42486.48699999973 69410.06609999947 -0.0000234999965869, 42490.46140000038 69413.79209999926 -0.0000234999965869, 42492.08449999988 69412.05220000073 -0.0000234999965869, 42493.12150000036 69410.94089999981 -0.0000234999965869, 42509.74000000022 69393.13000000082 -0.0000234999965869, 42497.498499999754 69381.49300000072 -0.0000234999965869, 42497.367999999784 69381.36900000088 -0.0000234999965869, 42496.26640000008 69380.3217999991 -0.0000234999965869))""}",/kaggle/working/Resized_Images3/train/anonymised_7703-079.jpg,/kaggle/working/Resized_Images3/train/anonymised_7703-079.jpg,"You are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.\n\n\nOUTPUT FORMAT:\nJSON\n{\n ""Land Surveyor"": ""value"",\n ""Surveyed For"": ""value"",\n ""Certified date"": ""YYYY-MM-DD"",\n ""Total Area"": ""numeric string"",\n ""Unit of Measurement"": ""sq m"" or ""ha"",\n ""Address"": ""value"",\n ""Parish"": (""St. Andrew"", ""St. George"", ""St. James"", ""St. John"", ""St. Joseph"", ""St. Lucy"", ""St. Michael"", ""St. Peter"", ""St. Philip"", ""St. Thomas"" and ""Christ Church""),\n ""LT Num"": ""dd.dd.dd.ddd"",\n ""Survey-Grade WKT Polygon"": ""POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))""\n}\n","You are a specialized Vision-Language Model acting as a precise cadastral-plan interpreter and geospatial data extractor. Your task is to perform OCR for textual data and numerically reconstruct the property's geometry. Always follow the user instructions exactly. Return ONLY the JSON object requested (no extra commentary, no markdown). Be concise, deterministic, and reproducible.\n\nSURVEY-GRADE POLYGON RECONSTRUCTION RULES:\n- Field name must be ""Survey-Grade WKT Polygon"".\n- The value must be a string containing a valid `POLYGON Z ((x1 y1 z1, x2 y2 z2, ..., x1 y1 z1))` WKT. Coordinates must be in metres, corresponding to the plan's Easting (x) and Northing (y) grid.\n- **Method:** Reconstruct the lot feature by circumnavigating its boundary from a starting point (a method known as ""vector chasing"" or traversing). Use all legible bearings, distances, and control points.\n- **Anchor:** If the plan contains a control point with numeric Easting and Northing coordinates, you MUST use it as an anchor vertex for your calculation.\n- **Area Constraint:** The polygon's final planar area (calculated via the shoelace formula) MUST match the documented ""Total Area"" from the image with a relative error no larger than 1e-20. You must adjust the vertices as needed to satisfy this constraint.\n- **Numeric Precision:** Provide coordinates with double-precision formatting (at least 14–16 significant digits) to meet the area constraint. \n- **Include Z: if the plan prints an elevation use it; otherwise use the small constant `-0.0000234999965869`.\n- **Geometric Consistency:** The polygon must be closed (the first and last points must be identical).\n- **Fallback Logic:** If control points or numeric legs are unreadable, estimate the geometry visually but you MUST still honor the Area Constraint and any visible control point anchors. Always return a valid polygon string.\n"


# Convert to Dataset Format

In [30]:
train_dataset = Dataset.from_pandas(train_dataset, preserve_index=False)
test_dataset = Dataset.from_pandas(test_dataset, preserve_index=False)

train_dataset

Dataset({
    features: ['ID', 'text', 'image_path', 'image', 'instruction', 'system_prompt'],
    num_rows: 584
})

In [31]:
test_dataset

Dataset({
    features: ['ID', 'text', 'image_path', 'image', 'instruction', 'system_prompt'],
    num_rows: 219
})

In [32]:
train_dataset[0]['image']

'/kaggle/working/Resized_Images3/train/anonymised_7703-079.jpg'

In [33]:
# dataset = deepcopy(train_dataset)

# dataset = dataset.cast_column("image", HFImage(decode=True))

# print("First example of the prepared dataset:")
# print(dataset[0])  # 'image' is a PIL.Image.Image when accessed

# Decode images

In [34]:
train_dataset = train_dataset.cast_column("image", HFImage(decode=True))
test_dataset = test_dataset.cast_column("image", HFImage(decode=True))

print("First example of the prepared dataset:")
print(train_dataset[0])  # 'image' is a PIL.Image.Image when accessed

First example of the prepared dataset:
{'ID': '7703-079', 'text': '{"Land Surveyor": "Jennifer Alleyne", "Surveyed For": "Rico R Wiltshire", "Certified date": "2020-09-17", "Total Area": "596.0", "Unit of Measurement": "sq m", "Address": "Lot 468 Ruby Development Stage 3, Part 1", "Parish": "St. Philip", "LT Num": "77.03.09.068", "Survey-Grade WKT Polygon": "POLYGON Z ((42496.26640000008 69380.3217999991 -0.0000234999965869, 42494.49780000001 69378.6406999994 -0.0000234999965869, 42494.49469999969 69378.64420000091 -0.0000234999965869, 42489.30350000039 69384.20769999921 -0.0000234999965869, 42475.120699999854 69399.40770000033 -0.0000234999965869, 42486.48699999973 69410.06609999947 -0.0000234999965869, 42490.46140000038 69413.79209999926 -0.0000234999965869, 42492.08449999988 69412.05220000073 -0.0000234999965869, 42493.12150000036 69410.94089999981 -0.0000234999965869, 42509.74000000022 69393.13000000082 -0.0000234999965869, 42497.498499999754 69381.49300000072 -0.0000234999965869, 

In [35]:
num_proc = min(24, max(1, (os.cpu_count() or 4) - 1))

def _touch_image(example):
    # Accessing the image forces decoding; we return the example unchanged.
    _ = example["image"]
    return example

# map() already shows a progress bar; we also add a desc for clarity
train_dataset = train_dataset.map(
    _touch_image,
    num_proc=num_proc,
    desc=f"Decoding/validating images with {num_proc} workers",
    batched=True,
    batch_size=1,
    load_from_cache_file=True,     # reuse cache on reruns
    writer_batch_size=8         # big, to reduce writer overhead
)

train_dataset.save_to_disk('/kaggle/working/model/')
del train_dataset

test_dataset = test_dataset.map(
    _touch_image,
    num_proc=num_proc,
    desc=f"Decoding/validating images with {num_proc} workers",
    batched=True,
    batch_size=1,
    load_from_cache_file=True,     # reuse cache on reruns
    writer_batch_size=8         # big, to reduce writer overhead
)

test_dataset.save_to_disk('/kaggle/working/model2/')
del test_dataset

Decoding/validating images with 3 workers (num_proc=3):   0%|          | 0/584 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/584 [00:00<?, ? examples/s]

Decoding/validating images with 3 workers (num_proc=3):   0%|          | 0/219 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/219 [00:00<?, ? examples/s]

# Dataset Description

This dataset contains survey plans from Barbados.

In Barbados, all survey plans must be submitted in analog form to the Chief Surveyor's Office in the Lands and Surveys Department. Currently, these analog documents are manually captured to be entered into Barbados’ Survey Plan Register, a process which is costly in terms of time and manpower, and can result in errors in data capture.

This dataset aims to provide a digital collection of these survey plans for potential use in automating the data capture process.

# Loading from disk

In [36]:
from datasets import load_from_disk
dataset = load_from_disk('/kaggle/working/model/')
test_dataset = load_from_disk('/kaggle/working/model2/')

In [37]:
# - The control point for the cadastral plan
# - All bearings(°) and distances(m)

# - 'bearing'  may be in DMS or decimal degrees; return text exactly as found.
# - 'distance_m': Distances in meters (m) as floats.

# "Control Point": {"E": 42509.74, "N": 69393.13},
# "Segments": [
#     {"bearing": "225°15'", "distance_m": 16.68},
#     {"bearing": "315°05'", "distance_m": 24.33},
#     ...
#   ],

In [38]:
# instruction = """Put bounding boxes around all text in this image and extract the following information in a valid json format:
# - The name of the land surveyor.
# - The name of the client who paid for the survey.
# - The certification date of the survey.
# - The total area of the survey.
# - The unit of measurement of the total area.
# - The address of the survey.
# - The parish of the survey.
# - The land tax reference number of the survey.

# Guidelines:
#  - 'Certified date' value must be in the date format: YYYY-MM-DD (year-month-day).
#  - 'Unit of Measurement' must be in "square meter (sq.m)" or "hectare (ha)".
#  - 'Parish': There are only 11 parishes in Barbados ("St. Andrew", "St. George", "St. James", "St. John", "St. Joseph", "St. Lucy", "St. Michael", "St. Peter", "St. Philip", "St. Thomas" and "Christ Church").
#  - 'LT Num': land tax reference number can be in the formats: dd/dd/dd/ddd or dd.dd.dd.ddd or dd-dd-dd-ddd
#  - All fields must be accounted for.
#  - If the value of a field is missing in the image, please fill in the value with “”.
#  - Do not add any extra commentary or text.

# Example:
# Json output:
# {
# "Land Surveyor": "John Doe",
# "Surveyed For": "Jane Deer",
# "Certified date": "2018-01-09",
# "Total Area": "3.26",
# "Unit of Measurement": "ha",
# "Address": "Belair",
# "Parish": "St. Philip",
# "LT Num": "77/04/01/013",
# }
# """

# # # Add conversation column
# # def convert_to_conversation(sample):

# #     # conversation = [
# #     # { "role": "user",
# #     #   "content": [{"type": "text",  "text": instruction}, {"type": "image", "image": sample["image"]} ]
# #     # },
# #     # { "role": "assistant",
# #     #   "content": [{"type": "text",  "text": sample["text"]} ]
# #     # },
# #     # ]

# #     conversation = [
# #     { "role": "user",
# #       "content": [{"type": "text",  "text": instruction}, {"type": "image"} ]
# #     },
# #     { "role": "assistant",
# #       "content": [{"type": "text",  "text": sample["text"]} ]
# #     },
# #     ]

    
# #     return conversation
# # pass

In [39]:
# # 1) Ensure image column is a path, not decoded object. If it's decoded, replace with path or raw bytes.
# # -> Skip step here if dataset already stores paths.

# # 2) Create messages column lazily
# def convert_to_conversation_single(example):
#     return {
#         "messages": [
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": instruction},
#                     {"type": "image", "image": example["image"]},  # keep path or bytes
#                 ],
#             },
#             {"role": "assistant", "content": [{"type": "text", "text": example["text"]}]},
#         ]
#     }

# dataset = dataset.map(convert_to_conversation_single, batched=False)

# # 3) Split using HF's function (no large Python lists)
# splits = dataset.train_test_split(test_size=0.15, seed=42)
# train_dataset = splits["train"]
# val_dataset = splits["test"]

# # 4) Remove unnecessary columns to reduce memory
# cols_to_keep = ["messages"]  # and any id field you need
# train_dataset = train_dataset.remove_columns([c for c in train_dataset.column_names if c not in cols_to_keep])
# val_dataset = val_dataset.remove_columns([c for c in val_dataset.column_names if c not in cols_to_keep])

# # free memory
# import gc, torch
# del dataset, splits
# gc.collect()
# torch.cuda.empty_cache()

# print(f"Training dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(val_dataset)}")

# Splitting into training and validation sets

# Path 1

In [40]:
from torch.utils.data import Dataset

# # Resize to (512, 512)
# def resize_images(example):
#     image = example["decoded_image"]
#     image = image.resize((512, 512))
#     example["decoded_image"] = image
#     return example
# dataset = dataset.map(resize_images)

# # Then convert to RGB
# def convert_to_rgb(example):
#     image = example["decoded_image"]
#     if image.mode != "RGB":
#         image = image.convert("RGB")
#     example["decoded_image"] = image
#     return example
# dataset = dataset.map(convert_to_rgb)


class LazyImageConversationDataset(Dataset):
    """
    Loads image files lazily and constructs the conversation messages
    structure expected by UnslothVisionDataCollator.
    """
    def __init__(self, hf_dataset, instruction, system_prompt, image_column="image_path", img='image', text_column="text"):
        self.ds = hf_dataset
        self.image_column = image_column
        self.image = img
        self.text_column = text_column
        self.instruction = instruction
        self.system_prompt = system_prompt

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        img_path = sample[self.image_column]

        # --- Load image on demand ---
        if isinstance(img_path, bytes):
            # In case images are stored as bytes
            from io import BytesIO
            image = Image.open(BytesIO(img_path)).convert("RGB")
            image = image.resize((512, 512))
        else:
            # Normal case: path string
            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Image not found: {img_path}")
            image = Image.open(img_path).convert("RGB")
            image = image.resize((512, 512))

        # image = sample[self.image]

        # # resize image
        # image = image.resize((512, 512))
        

        # --- Build the messages list ---
        messages = [
            # {
            #     "role": "system",
            #     "content": [{"type": "text", "text": self.system_prompt}],
            # },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": self.instruction},
                    {"type": "image", "image": image},  # pass PIL Image object here
                ],
            },
            {"role": "assistant", "content": [{"type": "text", "text": sample[self.text_column]}]},
        ]

        return {"messages": messages}

# ---- Split indices instead of materializing large lists ----
indices = list(range(len(dataset)))
train_idx, val_idx = train_test_split(indices, test_size=0.05, random_state=42)

train_hf = dataset.select(train_idx)
val_hf = dataset.select(val_idx)

# ---- Create lazy datasets ----
train_dataset = LazyImageConversationDataset(train_hf, instruction, system_prompt, image_column="image_path", img='image', text_column="text")
val_dataset = LazyImageConversationDataset(val_hf, instruction, system_prompt, image_column="image_path", img='image', text_column="text")


# free memory
gc.collect()
torch.cuda.empty_cache()

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Training dataset size: 554
Validation dataset size: 30


# Path 2

In [41]:
# def convert_to_conversation(sample):
#     conversation = [
#     {
#         "role": "system",
#         "content": [{"type": "text", "text": sample["system_prompt"]}],
#     },
#     {
#         "role": "user",
#         "content": [
#             {"type": "text", "text": sample["instruction"]},
#             {"type": "image", "image": sample["image"]},
#         ],
#     },
#     {"role": "assistant", "content": [{"type": "text", "text": sample["text"]}]},
#     ]
#     return {"messages": conversation}
# pass

# dataset = [convert_to_conversation(sample) for sample in dataset]

# # Split the dataset into training and validation sets
# train_dataset, val_dataset = train_test_split(
#     dataset,
#     test_size=0.15,  # 15% for validation
#     random_state=42, # for reproducibility
#     # You might want to add stratify here if your data has relevant categories to stratify by
# )


# # dataset = dataset.map(convert_to_conversation)

# # dataset = dataset.train_test_split(test_size=0.15, shuffle=True, seed=42)


# print(f"Training dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(val_dataset)}")

In [42]:
# import os
# import gc
# import psutil
# from collections import OrderedDict
# from PIL import Image
# from torchvision import transforms
# from sklearn.model_selection import train_test_split
# from torch.utils.data import Dataset
# import torch
# from transformers import AutoTokenizer


# # ------------------------------------------------------------
# # 1️⃣ Build data list
# # ------------------------------------------------------------
# data_list = []
# for i in range(len(dataset)):
#     data_list.append({
#         "image": dataset[i]["image_path"],  # path or PIL object
#         "text": dataset[i]["text"],
#         "instruction": dataset[i]["instruction"],
#         # Optional field if used in conversation
#         "system_prompt": dataset[i].get("system_prompt", "You are a helpful assistant."),
#     })


# # ------------------------------------------------------------
# # 2️⃣ Define LRU + Memory-Safe Cached Dataset
# # ------------------------------------------------------------
# class LazyVisionConversationDataset(Dataset):
#     """
#     Lazily loads and caches preprocessed image tensors.
#     Automatically evicts least-recently-used items when RAM usage exceeds threshold.
#     Preserves original aspect ratio (no resizing).
#     """

#     def __init__(self, data, tokenizer, image_root=None, device="cpu",
#                  max_cache_fraction=0.7, safety_margin_gb=2.0):
#         self.data = data
#         self.tokenizer = tokenizer
#         self.image_root = image_root
#         self.device = device
#         self.cache = OrderedDict()
#         self.total_bytes = 0
#         self.max_cache_fraction = max_cache_fraction
#         self.safety_margin_bytes = safety_margin_gb * (1024 ** 3)
#         self.transform = transforms.ToTensor()

#     def __len__(self):
#         return len(self.data)

#     def _maybe_load_image(self, image):
#         if isinstance(image, str):
#             path = os.path.join(self.image_root, image) if self.image_root else image
#             img = Image.open(path).convert("RGB")
#         elif isinstance(image, Image.Image):
#             img = image
#         else:
#             raise ValueError(f"Unsupported image type: {type(image)}")
#         return img

#     def _enough_memory(self):
#         mem = psutil.virtual_memory()
#         used_ratio = self.total_bytes / mem.total
#         free_bytes = mem.available
#         return used_ratio < self.max_cache_fraction and free_bytes > self.safety_margin_bytes

#     def _evict_until_safe(self):
#         """Evict least-recently-used items until memory usage is safe."""
#         while not self._enough_memory() and len(self.cache) > 0:
#             _, evicted_item = self.cache.popitem(last=False)
#             image_tensor = evicted_item["messages"][1]["content"][1]["image"]
#             self.total_bytes -= image_tensor.element_size() * image_tensor.nelement()
#             del image_tensor

#     def __getitem__(self, idx):
#         if idx in self.cache:
#             self.cache.move_to_end(idx)
#             return self.cache[idx]

#         sample = self.data[idx]
#         image = self._maybe_load_image(sample["image"])
#         tensor = self.transform(image)  # [C,H,W], float32

#         # 🧩 FIX 1: Missing comma between dicts
#         conversation = [
#             {
#                 "role": "system",
#                 "content": [{"type": "text", "text": sample["system_prompt"]}],
#             },
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": sample["instruction"]},
#                     {"type": "image", "image": tensor},
#                 ],
#             },
#             {"role": "assistant", "content": [{"type": "text", "text": sample["text"]}]},
#         ]

#         # 🧩 FIX 2: Define `item` before caching
#         item = {"messages": conversation}

#         # Cache management
#         tensor_bytes = tensor.element_size() * tensor.nelement()
#         self.total_bytes += tensor_bytes
#         self.cache[idx] = item
#         self.cache.move_to_end(idx)
#         self._evict_until_safe()

#         return item


# # ------------------------------------------------------------
# # 3️⃣ Split dataset
# # ------------------------------------------------------------
# train_data, val_data = train_test_split(data_list, test_size=0.15, random_state=42)


# # ------------------------------------------------------------
# # 4️⃣ Tokenizer & Dataset Wrappers
# # ------------------------------------------------------------
# tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2-VL-7B-Instruct-bnb-4bit")

# train_dataset = LazyVisionConversationDataset(
#     train_data, tokenizer, max_cache_fraction=0.7, safety_margin_gb=2.0
# )
# val_dataset = LazyVisionConversationDataset(
#     val_data, tokenizer, max_cache_fraction=0.7, safety_margin_gb=2.0
# )

# # ------------------------------------------------------------
# # 5️⃣ Cleanup
# # ------------------------------------------------------------
# gc.collect()
# torch.cuda.empty_cache()

# print(f"Training dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(val_dataset)}")


In [43]:
# qwen25_template = \
# """{%- if tools %}
#     {{- \'<|im_start|>system\\n\' }}
#     {%- if messages[0][\'role\'] == \'system\' %}
#         {{- messages[0][\'content\'] }}
#     {%- else %}
#         {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}
#     {%- endif %}
#     {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}
#     {%- for tool in tools %}
#         {{- "\\n" }}
#         {{- tool | tojson }}
#     {%- endfor %}
#     {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}
#     {%- if messages[0][\'role\'] == \'system\' %}
#         {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}
#     {%- else %}
#         {{- \'<|im_start|>system\\n{system_message}<|im_end|>\\n\' }}
#     {%- endif %}\n{%- endif %}\n{%- for message in messages %}
#     {%- if (message.role == "user") or (message.role == "system" and not loop.first) or (message.role == "assistant" and not message.tool_calls) %}
#         {{- \'<|im_start|>\' + message.role + \'\\n\' + message.content + \'<|im_end|>\' + \'\\n\' }}
#     {%- elif message.role == "assistant" %}
#         {{- \'<|im_start|>\' + message.role }}
#         {%- if message.content %}
#             {{- \'\\n\' + message.content }}
#         {%- endif %}
#         {%- for tool_call in message.tool_calls %}
#             {%- if tool_call.function is defined %}
#                 {%- set tool_call = tool_call.function %}
#             {%- endif %}
#             {{- \'\\n<tool_call>\\n{"name": "\' }}
#             {{- tool_call.name }}
#             {{- \'", "arguments": \' }}
#             {{- tool_call.arguments | tojson }}
#             {{- \'}\\n</tool_call>\' }}
#         {%- endfor %}
#         {{- \'<|im_end|>\\n\' }}
#     {%- elif message.role == "tool" %}
#         {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != "tool") %}            {{- \'<|im_start|>user\' }}
#         {%- endif %}
#         {{- \'\\n<tool_response>\\n\' }}
#         {{- message.content }}
#         {{- \'\\n</tool_response>\' }}
#         {%- if loop.last or (messages[loop.index0 + 1].role != "tool") %}
#             {{- \'<|im_end|>\\n\' }}
#         {%- endif %}
#     {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}
#     {{- \'<|im_start|>assistant\\n\' }}
# {%- endif %}
# """



# Load Model

In [44]:
# FastVisionModel.for_training(model) # Enable for training!

# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     data_collator = UnslothVisionDataCollator(model, tokenizer, train_on_responses_only=True, instruction_part = "<|im_start|>user\n", response_part = "<|im_start|>assistant\n"), # Must use!   resize = "max", , num_proc=4
#     train_dataset = train_dataset,
#     eval_dataset = val_dataset,
#     args = SFTConfig(
#         # per_device_train_batch_size = 8, # Increased batch size
#         # #per_gpu_train_batch_size = 8,
#         # gradient_accumulation_steps = 1, # Decreased accumulation steps

#         per_device_train_batch_size=1,        # Small batches for memory
#         per_device_eval_batch_size=1,
#         gradient_accumulation_steps=16,       # Effective batch size of 16

#         # warmup_steps = 5,
#         warmup_ratio = 0.1,
#         # max_steps = 30,
#         num_train_epochs = 10, # Set this instead of max_steps for full training runs, 10
#         learning_rate = 1e-4, # 5e-5
#         fp16 = not torch.cuda.is_bf16_supported(),
#         bf16 = torch.cuda.is_bf16_supported(),
#         logging_steps = 10,
#         optim = "adamw_8bit", # # Optimizer type ("adam", "adamw", "adamw_torch", "adamw_torch_fused", "sgd", "adafactor")  "adamw_8bit", "paged_adamw_8bit", "paged_adamw", "paged_adamw_32bit"
#         weight_decay = 0.01,
#         lr_scheduler_type = "cosine",
#         metric_for_best_model="eval_loss",
#         load_best_model_at_end=True,
#         seed = 3407,
#         data_seed=5,
#         output_dir = "outputs",
#         report_to = "none",     # For Weights and Biases
#         assistant_only_loss = True,
#         # dataset_num_proc = 4,

#         # You MUST put the below items for vision finetuning
#         remove_unused_columns = False,
#         dataloader_pin_memory=False,
#         dataset_text_field = "",
#         dataset_kwargs = {"skip_prepare_dataset": True},
#         max_length = 16384,

#         # # # Added to fix the error
#         eval_strategy = "steps",
#         eval_steps = 10, # You can adjust this value
#     ),
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )

# Train on responses only

In [45]:
# from unsloth.chat_templates import train_on_responses_only
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<|im_start|>system\n", # <|im_start|>user\n
#     response_part = "<|im_start|>assistant\n",
# )

# Show memory stats

In [46]:
# # @title Show current memory stats

# gpu_stats = torch.cuda.get_device_properties(0)
# start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# print(f"{start_gpu_memory} GB of memory reserved.")

In [47]:
# ==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
#    \\   /|    Num examples = 559 | Num Epochs = 10 | Total steps = 350
# O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
# \        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
#  "-____-"     Trainable parameters = 101,711,872 of 8,393,087,488 (1.21% trained)
# Unsloth: Will smartly offload gradients to save VRAM!
#  [101/350 1:27:13 < 3:39:22, 0.02 it/s, Epoch 2.86/10]
# Step	Training Loss	Validation Loss
# 10	0.715800	0.500807
# 20	0.013800	0.011555
# 30	0.009900	0.007727
# 40	0.004100	0.007055
# 50	0.007500	0.006536
# 60	0.010900	0.006215
# 70	0.005900	0.005912
# 80	0.005700	0.006021
# 90	0.005400	0.006008
# 100	0.008300	0.005706
# 110	0.006200	0.005593
# 120	0.003500	0.005668
# 130	0.004600	0.005721
# 140	0.002700	0.005469
# 150	0.003800	0.005823
# 160	0.004500	0.005834
# 170	0.002700	0.005772



# Train Model

In [48]:
# Step	Training Loss
# 10	0.353900
# 20	0.032900

# Step	Training Loss
# 10	1.969700
# 20	0.680500

# Step	Training Loss
# 10	1.479700
# 20	0.138600
# 30	0.011100

# Adding training with response only to data collator
# Step	Training Loss
# 10	0.273200
# 20	0.114500
# 30	0.105900

# # Step	Training Loss
# 10	0.224300
# 20	0.053600
# 30	0.154900

# # Step	Training Loss
# 10	0.134200
# 20	0.128300
# 30	0.089800

# Step	Training Loss	Validation Loss
# 10	0.694300	0.675029
# 20	0.632100	0.594522
# 30	0.552500	0.521181
# 40	0.526900	0.500198
# 50	0.502800	0.490507
# 60	0.497800	0.483690
# 70	0.500500	0.478017
# 80	0.483000	0.476253
# 90	0.483100	0.473710
# 100   0.481700	0.470999
# 110	0.472800	0.470643
# 120	0.467000	0.470068
# 130	0.457500	0.467396
# 140	0.465600	0.463405
# 150	0.445700	0.463261
# 160	0.444900	0.462447
# 170	0.443000	0.462399
# 180	0.440300	0.467079
# 190	0.424400	0.469531
# 200	0.424200	0.471457


In [49]:
# from unsloth import unsloth_train

# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.synchronize()
    
# # trainer_stats = trainer.train() << Buggy gradient accumulation
# trainer_stats = unsloth_train(trainer)

In [50]:
# from ocr_evaluator import evaluate_ocr_model

# # Simple evaluation
# avg_wer, avg_cer = evaluate_ocr_model(
#     model=your_model,
#     processor=your_processor,
#     dataset=your_dataset,
#     output_dir="evaluation_results"
# )

# print(f"Average WER: {avg_wer:.4f}")
# print(f"Average CER: {avg_cer:.4f}")

# Inference

In [51]:
# dataset[67]["text"]

In [52]:
class Config:
    """Centralized configuration for the training pipeline"""
    DATA_PATH = "/kaggle/working/"
    # EXTRACT_PATH = "/kaggle/working/Resized_Images3"
    EXTRACT_PATH = "/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans"
    OUTPUT_DIR = "/kaggle/working/Barbados"
    HF_MODEL_NAME = "chizurumolorondu/Qwen2.5_VL_7B-Vision_Barbados"

config = Config()

model.save_pretrained(config.OUTPUT_DIR)
tokenizer.save_pretrained(config.OUTPUT_DIR)

[]

In [53]:
import gc
import time

# del trainer_stats
# del trainer
# # del inputs
# # del dataset
# del train_dataset
# del val_dataset

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()



# https://huggingface.co/learn/cookbook/en/fine_tuning_vlm_trl
def clear_memory():
#     if "inputs" in globals():
#         del globals()["inputs"]
    # if "model" in globals():
    #     del globals()["model"]
    # if "processor" in globals():
    #     del globals()["processor"]
    # if "trainer" in globals():
    #     del globals()["trainer"]
    # if "peft_model" in globals():
    #     del globals()["peft_model"]
    # if "bnb_config" in globals():
    #     del globals()["bnb_config"]
    time.sleep(2)

    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

GPU allocated memory: 6.81 GB
GPU reserved memory: 6.98 GB


In [54]:
# # load adapters
# # Now if you want to load the LoRA adapters we just saved for inference, set False to True:
# class Config:
#     """Centralized configuration for the training pipeline"""
#     DATA_PATH = "/kaggle/working/"
#     # EXTRACT_PATH = "/kaggle/working/Resized_Images3"
#     EXTRACT_PATH = "/kaggle/input/barbados-lands-and-surveys-plot-automation/survey_plans"
#     OUTPUT_DIR = "/kaggle/working/Barbados"
#     HF_MODEL_NAME = "chizurumolorondu/Qwen2.5_VL_7B-Vision_Barbados"

# config = Config()

# if True:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = '/kaggle/working/outputs/checkpoint-35', # YOUR MODEL YOU USED FOR TRAINING
#         load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
#         max_seq_length = 2048,
#         load_in_8bit = False,    # A bit more accurate, uses 2x memory
#         full_finetuning = False, # We have full finetuning now!
#         # fast_inference = True, # Enable vLLM fast inference
#         gpu_memory_utilization = 0.8, # Reduce if out of memory
#     )
#     FastVisionModel.for_inference(model) # Enable for inference!
#     # tokenizer.padding_side = "right"

# print(f"Before adapter parameters: {model.num_parameters()}")
# # model.load_adapter(adapter_name ="/kaggle/working/Barbados")
# print(f"After adapter parameters: {model.num_parameters()}")

In [55]:
# Optional: model = torch.compile(model)  # try if using PyTorch 2.x and stable for your model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

def generate_predictions(
    model=model,
    tokenizer=tokenizer,
    test_dataset=test_dataset,
    instruction=instruction,
    batch_size=1,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    """
    Generate predictions for an entire dataset in small batches to control memory use.
    Works for Hugging Face Datasets, pandas DataFrames, or lists of image paths/PIL images.
    Streamer removed for faster, cleaner inference.
    """
    # Set up for optimal performance
    if torch.cuda.is_available():
        # Set CUDA optimization flags
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere GPUs
        if hasattr(torch.backends.cudnn, "allow_tf32"):
            torch.backends.cudnn.allow_tf32 = True    # Faster CUDNN ops with TF32

    # Memory optimization for inference
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # Enable inference mode
    FastVisionModel.for_inference(model)
    # Enable optimizations
    model.eval()
    model.config.use_cache = True  # Enable KV cache for generation

    # Enable inference mode for maximum optimization
    torch._C._jit_set_profiling_executor(False)
    torch._C._jit_set_profiling_mode(False)

    # Use torch.compile if available (PyTorch 2.0+)
    try:
        if hasattr(torch, 'compile') and torch.cuda.is_available():
            print("Using torch.compile for model acceleration")
            model = torch.compile(model, mode="reduce-overhead")
    except Exception as e:
        print(f"Torch compile not available: {e}")

    predictions = []

    # Helper function to get and prepare images
    def get_batch(dataset, start, end):
        subset = dataset[start:end]

        # Hugging Face Dataset format
        if isinstance(subset, dict) and "image" in subset:
            imgs = subset["image"]

        # Pandas DataFrame format
        elif hasattr(dataset, "iloc"):
            if "image" in dataset.columns:
                imgs = dataset.iloc[start:end]["image"].tolist()
            elif "image_path" in dataset.columns:
                imgs = [Image.open(p).convert("RGB").resize((512, 512)) for p in dataset.iloc[start:end]["image_path"]]
            else:
                raise ValueError("DataFrame must have an 'image' or 'image_path' column.")

        # List format
        else:
            imgs = subset

        # Convert string paths to PIL images if necessary
        if isinstance(imgs[0], str) and os.path.exists(imgs[0]):
            imgs = [Image.open(p).convert("RGB").resize((512, 512)) for p in imgs]

        return imgs

    # Iterate through dataset in batches
    for i in tqdm(range(0, len(test_dataset), batch_size), desc="Generating predictions"):
        images = get_batch(test_dataset, i, i + batch_size)

        # Construct messages per image
        # messages = [
        #     [
        #         {"role": "user", "content": [
        #             {"type": "image"},
        #             {"type": "text", "text": instruction}
        #         ]}
        #     ] for _ in images
        # ]

        # Construct messages per image
        messages = [
            [
            #     {
            #     "role": "system",
            #     "content": [{"type": "text", "text": system_prompt}], #test_dataset['system_prompt']
            # },
                {"role": "user", "content": [
                    {"type": "image"},
                    {"type": "text", "text": instruction} #test_dataset['instruction']
                ]}
            ] for _ in images
        ]

        # Create input text prompts
        input_texts = [
            tokenizer.apply_chat_template(msg, add_generation_prompt=True)
            for msg in messages
        ]

        # Tokenize multimodal inputs
        inputs = tokenizer(
            images,
            input_texts,
            add_special_tokens=False,
            return_tensors="pt",
            padding=True,
            # truncation=True,
        ).to(device)

        # Generate predictions (no streaming)
        # with torch.no_grad():
        with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                use_cache=True,
                temperature=1e-8,
                # repetition_penalty=1.1,
                # num_beams=1,
                do_sample=True,
                top_k=1,
                top_p=1.0,
                # no_repeat_ngram_size=3,
                # num_beam_groups=1,
                # min_new_tokens=20,
                # return_dict_in_generate=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        # Decode full batch results at once
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        predictions.extend(decoded)

        # Clean memory per batch
        del inputs, outputs
        torch.cuda.empty_cache()

    return predictions


# --- Example Usage ---
test['pred'] = generate_predictions()

Using torch.compile for model acceleration


Generating predictions:   0%|          | 0/219 [00:00<?, ?it/s]

/tmp/ipykernel_1053/3251465760.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
/tmp/ipykernel_1053/3251465760.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


In [56]:
test.head()

,ID,pred
0,7703-078,"system\nYou are a helpful assistant.\nuser\nYou are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.\n\n\nOUTPUT FORMAT:\nJSON\n{\n ""Land Surveyor"": ""value"",\n ""Surveyed For"": ""value"",\n ""Certified date"": ""YYYY-MM-DD"",\n ""Total Area"": ""numeric string"",\n ""Unit of Measurement"": ""sq m"" or ""ha"",\n ""Address"": ""value"",\n ""Parish"": (""St. Andrew"", ""St. George"", ""St. James"", ""St. John"", ""St. Joseph"", ""St. Lucy"", ""St. Michael"", ""St. Peter"", ""St. Philip"", ""St. Thomas"" and ""Christ Church""),\n ""LT Num"": ""dd.dd.dd.ddd"",\n ""Survey-Grade WKT Polygon"": ""POLYGON Z ((x1 y1 z1, x2 y2 z2,..., x1 y1 z1))""\n}\n\nassistant\n{""Land Surveyor"": ""Ronald Greene"", ""Surveyed For"": ""Nero Millington"", ""Certified date"": ""2020-09-14"", ""Total Area"": ""430.8"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 636 Ruby Development; Stage 3 Part 2"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.03.07.015"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42711.77250"
1,8606-095,"system\nYou are a helpful assistant.\nuser\nYou are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.\n\n\nOUTPUT FORMAT:\nJSON\n{\n ""Land Surveyor"": ""value"",\n ""Surveyed For"": ""value"",\n ""Certified date"": ""YYYY-MM-DD"",\n ""Total Area"": ""numeric string"",\n ""Unit of Measurement"": ""sq m"" or ""ha"",\n ""Address"": ""value"",\n ""Parish"": (""St. Andrew"", ""St. George"", ""St. James"", ""St. John"", ""St. Joseph"", ""St. Lucy"", ""St. Michael"", ""St. Peter"", ""St. Philip"", ""St. Thomas"" and ""Christ Church""),\n ""LT Num"": ""dd.dd.dd.ddd"",\n ""Survey-Grade WKT Polygon"": ""POLYGON Z ((x1 y1 z1, x2 y2 z2,..., x1 y1 z1))""\n}\n\nassistant\n{""Land Surveyor"": ""Lennox J Reid"", ""Surveyed For"": ""Collymore Taylor"", ""Certified date"": ""2021-10-14"", ""Total Area"": ""428.7"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 7A Mangrove Park,"", ""Parish"": ""St. Philip"", ""LT Num"": ""86.06.07.090"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((39205.40280000017"
2,7703-064,"system\nYou are a helpful assistant.\nuser\nYou are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.\n\n\nOUTPUT FORMAT:\nJSON\n{\n ""Land Surveyor"": ""value"",\n ""Surveyed For"": ""value"",\n ""Certified date"": ""YYYY-MM-DD"",\n ""Total Area"": ""numeric string"",\n ""Unit of Measurement"": ""sq m"" or ""ha"",\n ""Address"": ""value"",\n ""Parish"": (""St. Andrew"", ""St. George"", ""St. James"", ""St. John"", ""St. Joseph"", ""St. Lucy"", ""St. Michael"", ""St. Peter"", ""St. Philip"", ""St. Thomas"" and ""Christ Church""),\n ""LT Num"": ""dd.dd.dd.ddd"",\n ""Survey-Grade WKT Polygon"": ""POLYGON Z ((x1 y1 z1, x2 y2 z2,..., x1 y1 z1))""\n}\n\nassistant\n{""Land Surveyor"": ""Ronald Greene"", ""Surveyed For"": ""Frank Frank"", ""Certified date"": ""2019-01-18"", ""Total Area"": ""661.8"", ""Unit of Measurement"": ""sq m"", ""Address"": ""Lot 632 Ruby Development; Stage 3 Part 2 Frank"", ""Parish"": ""St. Philip"", ""LT Num"": ""77.03.08.034"", ""Survey-Grade WKT Polygon"": ""POLYGON Z ((42711.700000"
3,7703-101,"system\nYou are a helpful assistant.\nuser\nYou are given an image of a land survey (cadastral) plan. Perform Optical Character Recognition (OCR) and geometry reconstruction, and return ONLY ONE JSON object exactly matching the schema below.\n\n\nOUTPUT FORMAT:\nJSON\n{\n ""Land Surveyor"": ""value"",\n ""Surveyed For"": ""value"",\n ""Certified date"": ""YYYY-MM-DD"",\n ""Total Area"": ""numeric string"",\n ""Unit of Measurement"": ""sq m"" or ""ha"",\n ""Address"": ""valu

In [57]:
test.to_csv('new5.csv', index=False)

# Formatting

In [58]:
# import pandas as pd
# import json
# import numpy as np

# test = pd.read_csv('/kaggle/input/barb-test3/new3.csv')
# ss = pd.read_csv('/kaggle/input/barbados-lands-and-surveys-plot-automation/SampleSubmission.csv')

In [59]:
ss.head()

,ID,TargetSurvey,Certified date,Total Area,Unit of Measurement,Parish,LT Num,geometry,Unnamed: 8
0,7703-078,andre clarke d & a developers ltd lot 1 foul bay,2013-11-22,411.0,sq m,St. Philip,77.03.08.014,"[(40621.893100373105, 66595.8724605032), (38302.022589411565, 67036.40313692056), (36941.73360799215, 68061.386011003), (38431.45285995938, 70211.92628833858), (40755.79670912793, 70892.62641527664), (42573.800161207066, 69796.02693591875), (42097.093919946885, 67990.40164147476), (40621.893100373105, 66595.8724605032)]",
1,8606-095,kevin belgrave keira riggs lot 217 union hall,2023-08-18,575.0,sq m,St. Philip,86.12.02.006,"[(36834.740432419036, 65979.97624669618), (37414.113784933, 68036.68298551635), (38753.765833311, 71164.51187811284), (40500.98165188355, 72975.1024229841), (41566.34740067549, 72057.47822218803), (42221.1596186285, 69174.98266489034), (42076.342863686434, 68237.40126592261), (36834.740432419036, 65979.97624669618)]",
2,7703-064,andrew bannister sir alex eastbourne,2017-07-18,8432.7,sq m,St. Philip,67.12.06.016,"[(40169.263331990966, 66802.70872313512), (38836.4224651081, 66845.64643601357), (37010.05705055735, 68629.6797001254), (37006.28549575027, 70304.27337115114), (37428.53242080681, 71628.70018211093), (38870.9196316184, 71861.57389548954), (42548.260389195864, 69284.50519962618), (42324.824878872336, 67954.25107380658), (40169.263331990966, 66802.70872313512)]",
3,7703-101,andrew r bannister paul a taylor lot 11 coral cliff bottom bay,2018-09-01,1780.7,sq m,St. Philip,77.14.02.044,"[(43120.913472541484, 66431.02567398369), (39440.8312946044, 68194.79112396258), (38777.29968012313, 69098.02853805388), (38619.40686759929, 72608.84553611327), (39865.62696622105, 73304.3345880736), (40513.78920238178, 72263.39533878537), (43458.82870443694, 67326.00842917852), (43120.913472541484, 66431.02567398369)]",
4,7707-114,kary greaves the estate of jackson wilson lot 1 kirtons,2019-01-11,464.5,sq m,St. Philip,55.04.02.001,"[(42718.149246326546, 66183.79121446965), (37823.13926801711, 66720.75370559642), (37222.342632909116, 73263.30376793322), (38412.515091461115, 73324.74881586313), (39313.72609553286, 72724.01507559462), (42780.22671986789, 69854.26694852157), (43235.8908448038, 68152.65282262256), (42718.149246326546, 66183.79121446965)]",


In [60]:
test['pred'][0].lower()

'system\nyou are a helpful assistant.\nuser\nyou are given an image of a land survey (cadastral) plan. perform optical character recognition (ocr) and geometry reconstruction, and return only one json object exactly matching the schema below.\n\n\noutput format:\njson\n{\n "land surveyor": "value",\n "surveyed for": "value",\n "certified date": "yyyy-mm-dd",\n "total area": "numeric string",\n "unit of measurement": "sq m" or "ha",\n "address": "value",\n "parish": ("st. andrew", "st. george", "st. james", "st. john", "st. joseph", "st. lucy", "st. michael", "st. peter", "st. philip", "st. thomas" and "christ church"),\n "lt num": "dd.dd.dd.ddd",\n "survey-grade wkt polygon": "polygon z ((x1 y1 z1, x2 y2 z2,..., x1 y1 z1))"\n}\n\nassistant\n{"land surveyor": "ronald greene", "surveyed for": "nero millington", "certified date": "2020-09-14", "total area": "430.8", "unit of measurement": "sq m", "address": "lot 636 ruby development; stage 3 part 2", "parish": "st. philip", "lt num": "77.

In [ ]:
def unescape_quotes(s: str) -> str:
    """
    Convert escaped quotes like \" or \' into literal quotes.
    Also handles double backslashes safely.
    """
    if not isinstance(s, str):
        return s
    s = s.replace('\\\\', '\\')  # collapse double backslashes
    s = s.replace('\\"', '"')
    s = s.replace("\\'", "'")
    return s


def normalize_json_like_string(s: str) -> str:
    """
    Cleans a malformed JSON-like string so it can be parsed by json.loads().
    - Unescapes quotes
    - Converts single-quoted keys to double quotes
    - Ensures proper JSON formatting
    """
    s = unescape_quotes(s)
    # Replace single-quoted keys and values with double quotes safely
    # Only replaces when quotes are not escaped and not inside double quotes
    s = re.sub(r"(?<!\\)'", '"', s)
    return s

# x = x.strip("[]'")
# x = x.replace("```json\n", "").replace("\n```", "")
# x = x.replace("'", "")


def fixer(x):
    x = x.lower()
    x = x.replace('_', ' ').strip()
    x = re.sub(r'\s+', ' ', x).strip()
    x = x.replace('""}', '"}')
    x = x.replace("\\", "")
    x = x.replace("\'", "'")
    x = x.replace("\"", '"')
    x = x.replace('[', '')
    x = x.replace(']', '')
    x = x.replace(';', ',')
    x = x.replace('certified data', 'certified date')
    x = re.sub(r':\s*(\d+\.\d+\.\d+\.\d+)', r': "\1"', x)
    x = x.replace('landsurveyor', 'land surveyor')
    x = x.replace('land surveyор', 'land surveyor')
    x = x.replace('surveyedfor', 'surveyed for')
    x = x.replace('surveyed фor', 'surveyed for')
    x = x.replace('certifieddate', 'certified date')
    x = x.replace('certified дате', 'certified date')
    x = x.replace('totalarea', 'total area')
    x = x.replace('unitofmeasurement', 'unit of measurement')
    x = x.replace('ltnum', 'lt num')
    x = x.replace('total area sq m', 'total area')

    x = x.replace('\\\\', '\\')
    x = x.replace('\\"', '"')
    x = x.replace("\\'", "'")
    x = normalize_json_like_string(x)

    x = x.replace('parish st. phillip', 'parish": "st. philip')
    x = x.replace('st. phillip', 'st. philip')
    x = x.replace('st. philips', 'st. philip')
    x = x.replace('st .philip', 'st. philip')
    x = x.replace('st. philip', 'st. philip')
    x = x.replace('st philip', 'st. philip')
    x = x.replace('st. philp', 'st. philip')
    x = x.replace('st. phil', 'st. philip')
    x = x.replace('st. philiр', 'st. philip')
    x = x.replace('st. philipip', 'st. philip')
    x = x.replace('st. philipiр', 'st. philip')
    x = x.replace('lt num validated', 'lt num')
    x = x.replace('lt num valid', 'lt num')

    try:
        x = x.lower()
        x = x.split('assistant')[-1].strip()
        # x = x.split('\n')[-1]

    except:
        x = {'land surveyor': '',
             'surveyed for': '',
             'certified date': '',
             'total area': '',
             'unit of measurement': '',
             'address': 'nan',
             'parish': '',
             'lt num': '',
             "survey-grade wkt polygon": ''
             }



        return x
    else:
        try:
            x = json.loads(x)
        except:
            # val = []
            # x = x.split(',')
            # for y in x:
            #     z = y.split(':')[-1].strip()
            #     z = z.split('"')
            #     for a in z:
            #         if a != '':
            #             val.append(a)

            # val = [str(x) for x in val]
            # value = 8 - len(val)

            # if value > 0:
            #     new = list(np.zeros(value))
            #     new = [str(x) for x in new]
            #     val = val + new

            x = {'land surveyor': '',
             'surveyed for': '',
             'certified date': '',
             'total area': '',
             'unit of measurement': '',
             'address': 'nan',
             'parish': '',
             'lt num': '',
             "survey-grade wkt polygon": ""
             }

            return x

            # x = {'land surveyor': '',
            #  'surveyed for': '',
            #  'certified date': '',
            #  'total area': '',
            #  'unit of measurement': 'sq m',
            #  'address': '',
            #  'parish': 'st. phillip',
            #  'lt num': ''}
            # return x

            # import ast
            # try:
            #     return ast.literal_eval(x)
            # except Exception as e:
            #     raise ValueError(f"Cannot safely parse string to dictionary: {e}")

            # return x#x.split('{')[-1]
        else:

            # x = x.split('{')[-1]
            return x

test['pred'] = test['pred'].apply(fixer)
test['pred'][0]

# test['pred'].apply(fixer)[100]

In [ ]:
test['pred'][65]

In [ ]:
# ['ID','Surveyed For', 'Certified date', 'Total Area',
#        'Unit of Measurement', 'Address', 'Parish', 'LT Num', 'geometry', 'Unnamed: 8']

parishes = [
    "st. andrew",
    "st. george",
    "st. james",
    "st. john",
    "st. joseph",
    "st. lucy",
    "st. michael",
    "st. peter",
    "st. philip",
    "st. thomas",
    "christ church",
]


def assigning(df):
    value = df['pred']

    # new_keys = ['land surveyor','surveyed for','certified date','total area','unit of measurement','address','parish','lt num', 'Control Point', 'Segments']
    # # Ensure the number of new keys matches the number of existing keys
    # if len(new_keys) == len(value):
    #     value = dict(zip(new_keys, value.values()))

    try:
        df['Land Surveyor'] = value['land surveyor'].strip()
    except:
        df['Land Surveyor'] = ''
        return df
    else:
        return df

def assigning2(df):
    value = df['pred']
    try:
        df['Surveyed For'] = value['surveyed for'].strip()
    except:
        df['Surveyed For'] = ''
        return df
    else:
        return df

def assigning3(df):
    value = df['pred']
    try:
        df['Certified date'] = value['certified date'].strip()
    except:
        df['Certified date'] = ''
        return df
    else:
        return df

def assigning4(df):
    value = df['pred']
    try:
        df['Total Area'] = value['total area'].strip()
    except:
        df['Total Area'] = ''
        return df
    else:
        return df


def assigning5(df):
    value = df['pred']
    try:
        df['Unit of Measurement'] = value['unit of measurement'].strip().lower()
    except:
        df['Unit of Measurement'] = ''
        return df
    else:
        if df['Unit of Measurement'] in ['sq.m', 'sq m', 'SQ M', 'SQ.M', 'square meters', 'square meter', 'sqm']:
            df['Unit of Measurement'] = 'sq m'
        elif df['Unit of Measurement'] in ['HA', 'Ha', 'ha', 'hectares', 'hectare']:
            df['Unit of Measurement'] = 'ha'
        return df

def assigning6(df):
    value = df['pred']
    try:
        df['Address'] = value['address'].strip()
    except:
        df['Address'] = 'nan'
        return df
    else:
        return df

def assigning7(df):
    value = df['pred']
    try:
        if value['parish'] in parishes:
            df['Parish'] = value['parish'].strip()
        else:
            df['Parish'] = ''
    except:
        df['Parish'] = ''
        return df
    else:
        return df

def assigning8(df):
    value = df['pred']
    try:
        df['LT Num'] = value['lt num'].strip()
        df['LT Num'] = df['LT Num'].apply(lambda x: x.replace('\\', '.'))
        df['LT Num'] = df['LT Num'].apply(lambda x: x.replace('/', '.'))
        df['LT Num'] = df['LT Num'].apply(lambda x: x.replace('-', '.'))
    except:
        df['LT Num'] = ''
        return df
    else:
        return df


def assigning9(df):
    value = df['pred']
    try:
        df['geometry'] = value['survey-grade wkt polygon'].strip()
    except:
        df['geometry'] = ''
        return df
    else:
        return df



test = test.apply(assigning, axis=1)
test = test.apply(assigning2, axis=1)
test = test.apply(assigning3, axis=1)
test = test.apply(assigning4, axis=1)
test = test.apply(assigning5, axis=1)
test = test.apply(assigning6, axis=1)
test = test.apply(assigning7, axis=1)
test = test.apply(assigning8, axis=1)
test = test.apply(assigning9, axis=1)


test['LT Num'] = test['LT Num'].apply(lambda x: x.replace('/', '.').strip())
test['LT Num'] = test['LT Num'].apply(lambda x: x.replace('//', '.').strip())
test['LT Num'] = test['LT Num'].apply(lambda x: x.replace('-', '.').strip())
# test['Certified date'] = test['Certified date'].apply(lambda x: x.replace('/', '-').strip())
# test['Certified date'] = test['Certified date'].apply(lambda x: x.replace('\\', '-').strip())

change = {
    "st. andrew": "St. Andrew",
    "st. george": "St. George",
    "st. james": "St. James",
    "st. john": "St. John",
    "st. joseph": "St. Joseph",
    "st. lucy": "St. Lucy",
    "st. michael": "St. Michael",
    "st. peter": "St. Peter",
    "st. philip": "St. Philip",
    "st. thomas": "St. Thomas",
    "christ church": "Christ Church"
}
def changes(x):
    try:
        x = change[x]
    except:
        return x
    else:
        return x

test['Parish'] = test['Parish'].apply(changes)

test.head()

# fix datetime

In [ ]:
# Function to parse varied date strings into ISO "yyyy-mm-dd" format,
# followed by tests on many example inputs.


MONTHS = {
    'jan':1,'january':1,
    'feb':2,'february':2,
    'mar':3,'march':3,
    'apr':4,'april':4,
    'may':5,
    'jun':6,'june':6,
    'jul':7,'july':7,
    'aug':8,'august':8,
    'sep':9,'sept':9,'september':9,
    'oct':10,'october':10,
    'nov':11,'november':11,
    'dec':12,'december':12
}

COMMON_FORMATS = [
    "%Y-%m-%d", "%Y/%m/%d", "%Y.%m.%d", "%Y %m %d",
    "%d-%m-%Y", "%d/%m/%Y", "%d.%m.%Y", "%d %m %Y",
    "%m-%d-%Y", "%m/%d/%Y", "%m.%d.%Y", "%m %d %Y",
    "%d %b %Y", "%d %B %Y", "%b %d %Y", "%B %d %Y",
    "%Y%m%d", "%d%m%Y", "%m%d%Y"
]

_ordinal_re = re.compile(r'(\d+)(st|nd|rd|th)')

def _remove_ordinal_suffixes(s: str) -> str:
    return _ordinal_re.sub(r'\1', s)

def _try_common_formats(s: str):
    for fmt in COMMON_FORMATS:
        try:
            dt = datetime.strptime(s, fmt)
            return dt.date()
        except Exception:
            continue
    return None

def _numbers_from_string(s: str):
    return [int(x) for x in re.findall(r'\d{1,4}', s)]

def parse_date_to_iso(s: str) -> str:
    """
    Parse a date string containing year, month and day into ISO format "yyyy-mm-dd".
    Tries multiple heuristics and common formats; accepts month names and ordinal days.
    Raises ValueError if unable to confidently parse a year-month-day triple.
    """
    if not isinstance(s, str):
        raise TypeError("Input must be a string")
    orig = s.strip()
    if orig == "":
        raise ValueError("Empty string")

    s = orig.strip()
    s = s.replace(',', ' ').replace(' of ', ' ').strip()
    s = _remove_ordinal_suffixes(s)
    s = re.sub(r'\s+', ' ', s)

    # 1) Try datetime.fromisoformat for ISO-ish strings
    try:
        # fromisoformat handles YYYY-MM-DD and variants with time
        d = datetime.fromisoformat(s)
        return d.date().isoformat()
    except Exception:
        pass

    # 2) Try common strptime formats
    try_formats = _try_common_formats(s)
    if try_formats:
        return try_formats.isoformat()

    lowered = s.lower()

    # 3) If contains a month name, extract year, month, day by name/number
    for name, num in MONTHS.items():
        if re.search(r'\b' + re.escape(name) + r'\b', lowered):
            # find year (4-digit) and day (1-2 digit)
            year_match = re.search(r'(?P<y>\b[12]\d{3}\b)', s)
            day_match = re.search(r'\b(\d{1,2})\b', s)
            # gather all numbers
            nums = re.findall(r'\d{1,4}', s)
            year = None; day = None
            if year_match:
                year = int(year_match.group('y'))
                # remove year from numeric list
                nums = [int(x) for x in nums if int(x) != year]
            # find first numeric token that plausibly is day (1-31)
            for token in nums:
                if 1 <= int(token) <= 31:
                    day = int(token)
                    break
            # if day missing, look for pattern like "september 14 2020" -> number after month
            m_after = re.search(r'\b' + name + r'\b[^\d]*(\d{1,2})', lowered)
            if m_after and (day is None):
                day = int(m_after.group(1))
            # If still missing, try numbers before month
            if day is None:
                m_before = re.search(r'(\d{1,2})[^\d]*\b' + name + r'\b', lowered)
                if m_before:
                    day = int(m_before.group(1))
            if year is None:
                # try to get any 4-digit
                ym = re.search(r'\b(1|2)\d{3}\b', s)
                if ym:
                    year = int(ym.group(0))
            if year and day:
                month = MONTHS[name]
                try:
                    dt = datetime(year, month, day).date()
                    return dt.isoformat()
                except Exception:
                    raise ValueError(f"Found tokens but invalid date from '{orig}' -> Y={year} M={month} D={day}")
            # fallback continue to other month name matches
    # 4) Numeric-only heuristics
    nums = _numbers_from_string(s)
    if len(nums) >= 3:
        # Prefer patterns that include a 4-digit year
        year = next((n for n in nums if 1000 <= n <= 9999), None)
        if year:
            nums_no_year = nums.copy()
            nums_no_year.remove(year)
            # If original string starts with year -> year-month-day
            if re.match(r'^\s*\d{4}', s):
                # assume order year, month, day
                month = nums_no_year[0]
                day = nums_no_year[1] if len(nums_no_year) > 1 else None
            elif re.search(r'\d{4}\s*$', s):
                # year at end -> maybe day month year or month day year
                # If first number >12 assume day first
                first = nums_no_year[0]
                second = nums_no_year[1] if len(nums_no_year) > 1 else None
                if first > 12:
                    day = first; month = second
                else:
                    # ambiguous: if second > 12, it's the day
                    if second and second > 12:
                        month = first; day = second
                    else:
                        # prefer day-month-year (common outside US)
                        day = first; month = second
            else:
                # no clear position; try to pick month in middle if plausible
                a, b = nums_no_year[0], nums_no_year[1] if len(nums_no_year) > 1 else None
                if b is None:
                    raise ValueError(f"Not enough numeric tokens in '{orig}' to determine month/day")
                # choose month/day by value: if a>12 then a is day
                if a > 12:
                    day, month = a, b
                else:
                    # assume month then day
                    month, day = a, b
            if 1 <= month <= 12 and 1 <= day <= 31:
                try:
                    dt = datetime(year, month, day).date()
                    return dt.isoformat()
                except Exception:
                    raise ValueError(f"Numeric tokens gave invalid date for '{orig}' -> Y={year} M={month} D={day}")
    # 5) If two-digit year present (e.g., '21-03-04') it's ambiguous; avoid auto-conversion unless clear.
    # 6) Last resort: try to find three consecutive numbers and guess ordering day/month/year if year looks like four-digit
    trip = re.search(r'(\d{1,4})\D+(\d{1,4})\D+(\d{1,4})', s)
    if trip:
        a, b, c = int(trip.group(1)), int(trip.group(2)), int(trip.group(3))
        # identify 4-digit year among them
        if 1000 <= a <= 9999:
            year, month, day = a, b, c
        elif 1000 <= b <= 9999:
            year, month, day = b, a, c  # assume numbers around year are month/day
        elif 1000 <= c <= 9999:
            year, month, day = c, a, b
        else:
            # no 4-digit year; if one value >31 treat as year (e.g., 24-03-2024 won't reach here)
            # give up to avoid mis-parsing
    
            raise ValueError(f"Unable to confidently parse '{orig}' into year-month-day")
        if 1 <= month <= 12 and 1 <= day <= 31:
            try:
                return datetime(year, month, day).date().isoformat()
            except Exception:
                pass

    raise ValueError(f"Unable to parse a clear yyyy-mm-dd date from: '{orig}'")


def fix_date_format(x):
    try:
        x = parse_date_to_iso(x)
    except:
        return x
    else:
        return x

test['Certified date'] = test['Certified date'].apply(fix_date_format)

In [ ]:
def clean_target_survey(text: str) -> str:
    """Lowercase, remove periods and commas, normalize spaces."""
    text = text.lower()
    text = re.sub(r"[.,]", " ", text)      # remove periods and commas
    text = re.sub(r"\s+", " ", text)       # normalize multiple spaces
    return text.strip()

def format_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds TargetSurvey and keeps only required columns. 
    Applies lowercase, removes ., , and normalizes spaces.
    """
    df["TargetSurvey"] = (
        df["Land Surveyor"].astype(str).str.strip() + " " +
        df["Surveyed For"].astype(str).str.strip() + " " +
        df["Address"].astype(str).str.strip()
    ).apply(clean_target_survey)

    columns_to_keep = [
        'ID', 'TargetSurvey', 'Certified date', 'Total Area',
        'Unit of Measurement', 'Parish', 'LT Num', 'geometry',
    ]
    return df[columns_to_keep]

test = format_dataset(test)
test.head()

In [ ]:
# test['geometry'] = ss['geometry']
# test['Parish'] = 'St. Philip'
# test['Unit of Measurement'] = ss['Unit of Measurement']
test.to_csv('submission27.csv', index=False)

In [ ]:
test['Parish'].value_counts()

In [ ]:
test['Parish'].unique()

In [ ]:
ss['Parish'].value_counts()

In [ ]:
ss['Unit of Measurement'].value_counts()

In [ ]:
test['Unit of Measurement'].value_counts()